# LoRA Fine-Tuning for CLIP

This notebook fine-tunes CLIP using **LoRA** (Low-Rank Adaptation) on the food recipe dataset.

**Hypothesis H1:** The fine-tuned model improves text↔image alignment compared to the CLIP baseline, measured with retrieval metrics (R@K, MRR).

**This notebook (M1 LoRA):**
- Loads pre-trained CLIP model (no quantization)
- Applies LoRA adapters to trainable modules
- Fine-tunes on training set using CLIP contrastive loss (only positive pairs, in-batch negatives)
- Evaluates on test set with **same protocol as baseline** (by recipe_id)
- Saves adapters, logs, and results for comparison

**Inputs:**
- `fine-tuning-zone/datasets/train_pairs_augmented_with_negatives.csv` (for training, filtered to label==1)
- `fine-tuning-zone/datasets/test_pairs_positive.csv` (for evaluation)
- `fine-tuning-zone/images/` and `fine-tuning-zone/augmented_images/` (images)

**Outputs:**
- `fine-tuning-zone/experiments/lora/run_{run_id}/` — Complete run directory with:
  - `config.yaml` — Training configuration
  - `adapters/` — LoRA adapter weights
  - `training_logs.json` — Loss and timing logs
  - `results_lora.json` — Evaluation metrics
  - `examples_top5.json` — Qualitative examples


## 1. Setup and Configuration


In [ ]:
import os
import io
import json
import time
import sys
import math
import torch
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from datetime import datetime

# Check Python environment
print("=" * 60)
print("Python Environment Check")
print("=" * 60)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we're in a virtual environment
venv_path = Path(sys.executable).parent.parent
if (venv_path / "pyvenv.cfg").exists() or "venv" in str(sys.executable).lower():
    print(f"✓ Running in virtual environment: {venv_path}")
else:
    print(f"⚠ Not in a virtual environment")
    print(f"  Current Python: {sys.executable}")
    print(f"  If you have a venv, make sure Jupyter is using it!")

import numpy as np
import pandas as pd
import torch.nn.functional as F
from PIL import Image
import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv
from tqdm import tqdm
import yaml

# CLIP and LoRA imports
TRANSFORMERS_AVAILABLE = False
PEFT_AVAILABLE = False

try:
    from transformers import CLIPModel, CLIPProcessor
    TRANSFORMERS_AVAILABLE = True
    print("✓ transformers imported successfully")
except ImportError as e:
    print(f"✗ Failed to import transformers: {e}")

try:
    from peft import LoraConfig, get_peft_model, TaskType
    PEFT_AVAILABLE = True
    print("✓ peft imported successfully")
except ImportError as e:
    print(f"✗ Failed to import peft: {e}")
    TaskType = None

print("\n" + "=" * 60)
print("Library Status")
print("=" * 60)
if not TRANSFORMERS_AVAILABLE:
    print("  ✗ transformers: NOT AVAILABLE")
    print("    Install with: pip install transformers")
else:
    print("  ✓ transformers: AVAILABLE")
    
if not PEFT_AVAILABLE:
    print("  ✗ peft: NOT AVAILABLE")
    print("    Install with: pip install peft")
else:
    print("  ✓ peft: AVAILABLE")

if TRANSFORMERS_AVAILABLE and PEFT_AVAILABLE:
    print("\n✓ All required libraries are available!")
    print("  You can proceed with training.")
elif not TRANSFORMERS_AVAILABLE or not PEFT_AVAILABLE:
    print("\n✗ Some required libraries are missing!")
    print("  Please install missing libraries before continuing.")

# Load environment variables
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent

ENV_PATHS = [
    PROJECT_ROOT / "notebooks" / ".env",
    PROJECT_ROOT / "app" / ".env",
    PROJECT_ROOT / ".env",
]

env_loaded = False
for env_path in ENV_PATHS:
    if env_path.exists():
        load_dotenv(env_path)
        print(f"✓ Loaded .env from: {env_path}")
        env_loaded = True
        break

if not env_loaded:
    print("⚠ No .env file found, trying default load_dotenv()...")
    load_dotenv()

# MinIO Configuration
MINIO_USER = os.getenv("MINIO_USER")
MINIO_PASSWORD = os.getenv("MINIO_PASSWORD")
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")

# Bucket configuration
FINE_TUNING_BUCKET = "fine-tuning-zone"
DATASETS_PREFIX = "datasets"
IMAGES_PREFIX = "images"
AUGMENTED_IMAGES_PREFIX = "augmented_images"
EXPERIMENTS_PREFIX = "experiments"

# Input/Output paths
TRAIN_AUGMENTED_KEY = f"{DATASETS_PREFIX}/train_pairs_augmented_with_negatives.csv"
TEST_MANIFEST_KEY = f"{DATASETS_PREFIX}/test_pairs_positive.csv"  # Test set from baseline split

# Model configuration
MODEL_NAME = "openai/clip-vit-base-patch32"  # Same as baseline

def pick_device():
    """Pick device, testing if CUDA actually works (not just available)."""
    if not torch.cuda.is_available():
        return "cpu"
    try:
        _ = torch.randn(1, device="cuda")
        return "cuda"
    except Exception as e:
        print(f"⚠ CUDA visible pero no usable: {e}")
        return "cpu"

DEVICE = pick_device()

# LoRA configuration
LORA_CONFIG = {
    "r": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": TaskType.FEATURE_EXTRACTION if TaskType else "FEATURE_EXTRACTION",  # Use enum for compatibility
    "target_modules": ["q_proj", "k_proj", "v_proj", "out_proj"],  # Only attention modules (safer for CLIP)
}

# Training configuration
TRAINING_CONFIG = {
    "learning_rate": 1e-4,
    "num_epochs": 3,
    "batch_size": 2,  # Smaller batch for CPU training
    "gradient_accumulation_steps": 16,  # Effective batch size = 2 * 16 = 32
    "warmup_steps": 100,
    "save_steps": 500,
    "logging_steps": 50,
    "random_seed": 42,
}

# Evaluation configuration (same as baseline)
K_VALUES = [1, 5, 10]
COMPUTE_MRR = True

# Generate run ID
RUN_ID = f"lora_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"
RUN_DIR = f"{EXPERIMENTS_PREFIX}/lora/run_{RUN_ID}"

# Output paths
CONFIG_KEY = f"{RUN_DIR}/config.yaml"
ADAPTERS_DIR = f"{RUN_DIR}/adapters"
LOGS_KEY = f"{RUN_DIR}/training_logs.json"
RESULTS_KEY = f"{RUN_DIR}/results_lora.json"
EXAMPLES_KEY = f"{RUN_DIR}/examples_top5.json"

# Update config to include dataset info
TRAINING_CONFIG["train_dataset"] = TRAIN_AUGMENTED_KEY
TRAINING_CONFIG["test_dataset"] = TEST_MANIFEST_KEY
TRAINING_CONFIG["filter_applied"] = "label == 1 (positive pairs only)"

# Logit scale clamping constants (CLIP best practice)
LOGIT_SCALE_MIN = math.log(1/100)  # Minimum temperature: 0.01
LOGIT_SCALE_MAX = math.log(100)     # Maximum temperature: 100

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Device: {DEVICE}")
print(f"  Method: LoRA")
print(f"  LoRA r: {LORA_CONFIG['r']}, alpha: {LORA_CONFIG['lora_alpha']}")
print(f"  Training: {TRAINING_CONFIG['num_epochs']} epochs, LR={TRAINING_CONFIG['learning_rate']}")
print(f"  Batch size: {TRAINING_CONFIG['batch_size']} (effective: {TRAINING_CONFIG['batch_size'] * TRAINING_CONFIG['gradient_accumulation_steps']})")
print(f"  Run ID: {RUN_ID}")


## 2. Initialize MinIO Client


In [ ]:
# Initialize S3/MinIO client
session = boto3.session.Session(
    aws_access_key_id=MINIO_USER,
    aws_secret_access_key=MINIO_PASSWORD,
    region_name="us-east-1"
)
s3 = session.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    config=Config(signature_version="s3v4", s3={"addressing_style": "path"})
)

def ensure_bucket_exists(bucket: str) -> bool:
    """Create bucket if it doesn't exist."""
    try:
        s3.head_bucket(Bucket=bucket)
        return True
    except ClientError as e:
        error_code = e.response.get("Error", {}).get("Code", "")
        if error_code in ("404", "NoSuchBucket"):
            try:
                s3.create_bucket(Bucket=bucket)
                print(f"✓ Created bucket '{bucket}'")
                return True
            except ClientError as create_error:
                print(f"✗ Failed to create bucket '{bucket}': {create_error}")
                return False
        return False

# Verify buckets
print("Checking buckets...")
ensure_bucket_exists(FINE_TUNING_BUCKET)
print("✓ Buckets ready")


## 3. Load Training and Test Datasets


In [ ]:
def load_csv_from_minio(bucket: str, key: str) -> pd.DataFrame:
    """Load CSV file from MinIO into a DataFrame."""
    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(io.BytesIO(obj["Body"].read()))
        print(f"✓ Loaded {len(df)} rows from s3://{bucket}/{key}")
        return df
    except ClientError as e:
        print(f"✗ Failed to load s3://{bucket}/{key}: {e}")
        return pd.DataFrame()

# Load training dataset (augmented, with negatives)
print("Loading augmented training dataset...")
train_full_df = load_csv_from_minio(FINE_TUNING_BUCKET, TRAIN_AUGMENTED_KEY)

if train_full_df.empty:
    raise RuntimeError(f"Could not load training data from s3://{FINE_TUNING_BUCKET}/{TRAIN_AUGMENTED_KEY}")

print(f"\nFull training dataset shape: {train_full_df.shape}")
print(f"Columns: {list(train_full_df.columns)}")
if "label" in train_full_df.columns:
    print(f"\nLabel distribution:")
    print(train_full_df["label"].value_counts())
    
    # Filter only positive pairs (label=1)
    train_df = train_full_df[train_full_df["label"] == 1].copy()
    print(f"\n✓ Filtered to {len(train_df)} positive pairs (label=1)")
    print(f"  Removed {len(train_full_df) - len(train_df)} negative pairs")
else:
    # If no label column, assume all are positive
    train_df = train_full_df.copy()
    print(f"\n✓ No label column found, using all {len(train_df)} pairs as positive")

# Load test dataset (from baseline split, already filtered)
print("\nLoading test dataset...")
test_df = load_csv_from_minio(FINE_TUNING_BUCKET, TEST_MANIFEST_KEY)

if test_df.empty:
    raise RuntimeError(f"Could not load test data from s3://{FINE_TUNING_BUCKET}/{TEST_MANIFEST_KEY}")

print(f"\nTraining dataset:")
print(f"  Pairs: {len(train_df)}")
print(f"  Recipes: {train_df['recipe_id'].nunique()}")
print(f"  Images: {train_df['image_key'].nunique()}")
print(f"  Captions: {train_df['caption'].nunique()}")

print(f"\nTest dataset:")
print(f"  Pairs: {len(test_df)}")
print(f"  Recipes: {test_df['recipe_id'].nunique()}")
print(f"  Images: {test_df['image_key'].nunique()}")
print(f"  Captions: {test_df['caption'].nunique()}")


## 4. Helper Functions for Image Loading


In [ ]:
def load_image_from_minio(bucket: str, key: str) -> Optional[Image.Image]:
    """Load an image from MinIO."""
    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        img = Image.open(io.BytesIO(obj["Body"].read()))
        img.load()
        return img.convert("RGB")
    except Exception as e:
        return None

# Cache for image paths to avoid head_object per sample (performance optimization)
_image_path_cache: Dict[str, Tuple[str, str]] = {}

def build_image_path_cache(df: pd.DataFrame) -> Dict[str, Tuple[str, str]]:
    """
    Pre-build cache of image_key -> (bucket, full_key) to avoid head_object calls during training.
    This significantly improves throughput when loading images from MinIO.
    """
    print("Building image path cache...")
    cache = {}
    bucket = FINE_TUNING_BUCKET
    unique_keys = df["image_key"].unique()
    
    for image_key in tqdm(unique_keys, desc="Caching image paths"):
        # Check if it's already a full path
        if image_key.startswith("images/") or image_key.startswith("augmented_images/"):
            cache[image_key] = (bucket, image_key)
        else:
            # Try images/ first
            key = f"{IMAGES_PREFIX}/{image_key}"
            # Check if exists, if not try augmented_images/
            try:
                s3.head_object(Bucket=bucket, Key=key)
                cache[image_key] = (bucket, key)
            except ClientError as e:
                if e.response.get("Error", {}).get("Code") == "404":
                    # Try augmented_images/
                    key = f"augmented_images/{image_key}"
                    cache[image_key] = (bucket, key)
                else:
                    raise
    
    print(f"✓ Cached {len(cache)} image paths")
    return cache

def get_image_path_in_minio(image_key: str) -> Tuple[str, str]:
    """
    Determine bucket and full key for an image (uses cache if available).
    
    Images can be in:
    - fine-tuning-zone/images/...
    - fine-tuning-zone/augmented_images/...
    
    If cache is built, uses it. Otherwise falls back to head_object (slower).
    """
    # Use cache if available
    if image_key in _image_path_cache:
        return _image_path_cache[image_key]
    
    # Fallback: check if it's already a full path
    bucket = FINE_TUNING_BUCKET
    if image_key.startswith("images/") or image_key.startswith("augmented_images/"):
        key = image_key
    else:
        # Try images/ first
        key = f"{IMAGES_PREFIX}/{image_key}"
        # Check if exists, if not try augmented_images/
        try:
            s3.head_object(Bucket=bucket, Key=key)
        except ClientError as e:
            if e.response.get("Error", {}).get("Code") == "404":
                # Try augmented_images/
                key = f"augmented_images/{image_key}"
            else:
                raise
    
    return bucket, key


## 5. Load CLIP Model with LoRA


In [ ]:
if not TRANSFORMERS_AVAILABLE or not PEFT_AVAILABLE:
    raise ImportError("Required libraries not available. Install: pip install transformers peft")

print(f"Loading CLIP model with LoRA: {MODEL_NAME}")
print(f"Device: {DEVICE}")

# Load model
print("Loading model...")
model = CLIPModel.from_pretrained(MODEL_NAME).to(DEVICE)

# Load processor
processor = CLIPProcessor.from_pretrained(MODEL_NAME)

print("Model ready for training (LoRA)")

# Validate target_modules match actual CLIP module names (guardrail)
print("\nValidating target_modules against actual CLIP architecture...")
all_module_names = [name for name, _ in model.named_modules()]
target_modules = LORA_CONFIG["target_modules"]
matched_modules = []
for target in target_modules:
    matches = [name for name in all_module_names if target in name]
    if matches:
        matched_modules.extend(matches)
        print(f"  ✓ '{target}' matches: {matches[:3]}...")  # Show first 3
    else:
        print(f"  ⚠ '{target}' not found in model modules!")

if not matched_modules:
    print("\n⚠ WARNING: No target_modules matched! LoRA may not be applied.")
    print("Available attention-related modules (sample):")
    attn_modules = [name for name in all_module_names if "attn" in name.lower() or "proj" in name.lower()]
    for mod in attn_modules[:10]:  # Show first 10
        print(f"  - {mod}")
    print("  ... (use these to adjust target_modules)")

# Configure LoRA
lora_config = LoraConfig(
    r=LORA_CONFIG["r"],
    lora_alpha=LORA_CONFIG["lora_alpha"],
    lora_dropout=LORA_CONFIG["lora_dropout"],
    bias=LORA_CONFIG["bias"],
    task_type=LORA_CONFIG["task_type"],
    target_modules=LORA_CONFIG["target_modules"],
)

# Apply LoRA adapters
print("Applying LoRA adapters...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
print("\n" + "=" * 60)
print("Model Information")
print("=" * 60)
model.print_trainable_parameters()

# Check logit_scale trainability
print(f"\nlogit_scale requires_grad: {model.logit_scale.requires_grad}")
if not model.logit_scale.requires_grad:
    print("⚠ logit_scale is frozen. Enabling it for better fine-tuning (1 extra parameter, almost free).")
    model.logit_scale.requires_grad_(True)
    print("✓ logit_scale is now trainable")

# Get model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✓ Model loaded successfully")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Trainable ratio: {trainable_params/total_params*100:.2f}%")


## 6. Create DataLoader for Training


In [ ]:
from torch.utils.data import Dataset, DataLoader
import random

class CLIPDataset(Dataset):
    """Dataset for CLIP training with image-text pairs."""
    
    def __init__(self, df: pd.DataFrame, processor, s3_client, bucket: str):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.s3 = s3_client
        self.bucket = bucket
        self.failed_image_count = 0
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image
        bucket, key = get_image_path_in_minio(row["image_key"])
        img = load_image_from_minio(bucket, key)
        
        # Get caption
        caption = row["caption"]
        
        if img is None:
            # Return a placeholder if image fails to load
            self.failed_image_count += 1
            return {"image": None, "text": caption, "is_valid": False}
        
        return {
            "image": img,
            "text": caption, 
            "is_valid": True
        }

def collate_fn(batch):
    """Collate function for DataLoader."""
    # Filter out invalid samples (images that failed to load)
    batch = [b for b in batch if b.get("is_valid", True) and b["image"] is not None]
    
    if len(batch) == 0:
        return None  # Signal to skip this batch
    
    images = [item["image"] for item in batch]
    texts = [item["text"] for item in batch]
    
    # Process with CLIP processor
    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    
    # Filter to only include what CLIPModel.forward() accepts
    # CLIPModel expects: input_ids, attention_mask, pixel_values
    allowed_keys = {"input_ids", "attention_mask", "pixel_values"}
    return {k: v for k, v in inputs.items() if k in allowed_keys}

# Build image path cache (performance optimization: avoid head_object per sample)
print("Building image path cache for training set...")
_image_path_cache.update(build_image_path_cache(train_df))

# Create datasets and dataloaders
print("\nCreating datasets and dataloaders...")

train_dataset = CLIPDataset(train_df, processor, s3, FINE_TUNING_BUCKET)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=TRAINING_CONFIG["batch_size"],
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,  # Set to 0 to avoid issues with MinIO
)

print(f"✓ Created training dataloader")
print(f"  Total batches: {len(train_dataloader)}")
print(f"  Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"  Gradient accumulation steps: {TRAINING_CONFIG['gradient_accumulation_steps']}")
print(f"  Effective batch size: {TRAINING_CONFIG['batch_size'] * TRAINING_CONFIG['gradient_accumulation_steps']}")



## 7. Visualize Retrieval Results

Visualize the retrieval results to see how well the fine-tuned model performs:
- **Text → Image**: For each caption, show the top-5 retrieved images
- **Image → Text**: For each image, show the top-5 retrieved captions


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Use random seed from training config
RANDOM_SEED = TRAINING_CONFIG["random_seed"]

def visualize_text_to_image_retrieval(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_text: Dict[str, str],
    recipe_to_image_indices: Dict[str, List[int]],
    caption_to_idx: Dict[str, int],
    image_keys: List[str],
    image_key_to_recipe_id: Dict[str, str],
    n_examples: int = 3
):
    """
    Visualize Text → Image retrieval: for each recipe, show top-5 images.
    """
    
    # Sample random recipes
    unique_recipe_ids = test_df["recipe_id"].unique()
    sample_recipe_ids = pd.Series(unique_recipe_ids).sample(
        min(n_examples, len(unique_recipe_ids)),
        random_state=RANDOM_SEED
    )
    
    for recipe_idx, recipe_id in enumerate(sample_recipe_ids):
        recipe_id_str = str(recipe_id)
        # Get representative caption for this recipe
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        
        # Get caption embedding
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        
        text_emb = text_embeddings[text_idx]
        similarities = image_embeddings @ text_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]  # Top 5, highest first
        
        # Get ground truth: images from this recipe_id
        gt_image_indices = set(recipe_to_image_indices.get(recipe_id_str, []))
        
        # Create figure with more vertical space for captions
        fig = plt.figure(figsize=(16, 5))
        gs = gridspec.GridSpec(1, 6, width_ratios=[2, 1, 1, 1, 1, 1], hspace=0.3, bottom=0.15)
        
        # Show query caption
        ax0 = fig.add_subplot(gs[0])
        ax0.text(0.5, 0.5, f'Query:\n"{caption}"', 
                ha='center', va='center', fontsize=14, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
        ax0.axis('off')
        
        # Show top-5 images
        for i, img_idx in enumerate(top5_indices):
            img_key = image_keys[img_idx]
            bucket, key = get_image_path_in_minio(img_key)
            img = load_image_from_minio(bucket, key)
            
            if img is None:
                continue
            
            # Check if this image is from the ground truth recipe
            is_correct = img_idx in gt_image_indices
            
            # Get recipe_id for this image to show its representative caption
            img_recipe_id = image_key_to_recipe_id.get(img_key, "Unknown")
            # Note: Shows representative caption of the recipe (longest caption), not necessarily the exact caption for this image
            img_caption = recipe_to_caption_text.get(img_recipe_id, "Unknown")
            
            ax = fig.add_subplot(gs[i + 1])
            ax.imshow(img)
            ax.axis('off')
            
            # Color border: green if correct, red if incorrect
            border_color = 'green' if is_correct else 'red'
            border_width = 3 if is_correct else 2
            for spine in ax.spines.values():
                spine.set_edgecolor(border_color)
                spine.set_linewidth(border_width)
            
            # Title with similarity score
            similarity_score = similarities[img_idx]
            rank = i + 1
            title = f"Rank {rank}\n{similarity_score:.3f}"
            if is_correct:
                title += "\n✓ Correct"
            ax.set_title(title, fontsize=10, color=border_color, fontweight='bold')
            
            # Add recipe caption below the image
            # Truncate long captions for display
            display_caption = img_caption if len(img_caption) <= 40 else img_caption[:37] + "..."
            ax.text(0.5, -0.15, f'"{display_caption}"', 
                   ha='center', va='top', fontsize=9, 
                   transform=ax.transAxes,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'),
                   wrap=True)
        
        plt.suptitle(f'Text → Image Retrieval Example {recipe_idx + 1} (Recipe: {recipe_id_str})', 
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        print()

def visualize_image_to_text_retrieval(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    recipe_to_caption_text: Dict[str, str],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    unique_captions: List[str],
    n_examples: int = 3
):
    """
    Visualize Image → Text retrieval: for each image, show top-5 captions.
    """
    
    # Sample random images
    unique_images = test_df["image_key"].unique()
    sample_images = pd.Series(unique_images).sample(
        min(n_examples, len(unique_images)),
        random_state=RANDOM_SEED
    )
    
    for img_idx_example, img_key in enumerate(sample_images):
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        similarities = text_embeddings @ img_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]  # Top 5, highest first
        
        # Get ground truth: recipe_id for this image
        gt_recipe_id_str = image_key_to_recipe_id.get(img_key)
        # Use all captions for ground truth (consistent with evaluation)
        gt_caption_indices = set(recipe_to_all_caption_indices.get(gt_recipe_id_str, [])) if gt_recipe_id_str else set()
        
        # Load query image
        bucket, key = get_image_path_in_minio(img_key)
        query_img = load_image_from_minio(bucket, key)
        
        if query_img is None:
            continue
        
        # Create figure with more vertical space for recipe name
        fig = plt.figure(figsize=(16, 4))
        gs = gridspec.GridSpec(1, 6, width_ratios=[1, 1, 1, 1, 1, 1], hspace=0.3, bottom=0.15)
        
        # Show query image
        ax0 = fig.add_subplot(gs[0])
        ax0.imshow(query_img)
        ax0.axis('off')
        ax0.set_title('Query Image', fontsize=12, fontweight='bold')
        
        # Add recipe name below the query image
        if gt_recipe_id_str:
            recipe_caption = recipe_to_caption_text.get(gt_recipe_id_str, "Unknown")
            # Truncate long captions for display
            display_recipe = recipe_caption if len(recipe_caption) <= 40 else recipe_caption[:37] + "..."
            ax0.text(0.5, -0.15, f'Recipe: {gt_recipe_id_str}\n"{display_recipe}"', 
                   ha='center', va='top', fontsize=9, 
                   transform=ax0.transAxes,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'),
                   wrap=True)
        
        # Show top-5 captions
        for i, text_idx in enumerate(top5_indices):
            caption = unique_captions[text_idx]
            
            # Check if this caption is from the ground truth recipe
            is_correct = text_idx in gt_caption_indices
            
            ax = fig.add_subplot(gs[i + 1])
            ax.axis('off')
            
            # Color border: green if correct, red if incorrect
            border_color = 'green' if is_correct else 'red'
            border_width = 3 if is_correct else 2
            
            # Create text box
            similarity_score = similarities[text_idx]
            rank = i + 1
            text_content = f"Rank {rank}\n\n\"{caption}\"\n\nSimilarity: {similarity_score:.3f}"
            if is_correct:
                text_content += "\n\n✓ Correct"
            
            ax.text(0.5, 0.5, text_content,
                   ha='center', va='center', fontsize=10,
                   bbox=dict(boxstyle='round', facecolor='lightyellow', 
                            edgecolor=border_color, linewidth=border_width, alpha=0.8),
                   wrap=True)
        
        plt.suptitle(f'Image → Text Retrieval Example {img_idx_example + 1} (Recipe: {gt_recipe_id_str})', 
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        print()

# Visualization functions defined above
# They will be called after embeddings are generated (see section after evaluation)


In [ ]:
def clip_contrastive_loss(image_embeds, text_embeds, logit_scale):
    """
    CLIP contrastive loss with in-batch negatives.
    
    Args:
        image_embeds: Image embeddings (batch_size, embed_dim)
        text_embeds: Text embeddings (batch_size, embed_dim)
        logit_scale: Learnable temperature parameter
    
    Returns:
        loss: Contrastive loss
    """
    # Normalize embeddings
    image_embeds = F.normalize(image_embeds, dim=-1)
    text_embeds = F.normalize(text_embeds, dim=-1)
    
    # Compute logits
    logit_scale = logit_scale.exp()
    logits_per_text = logit_scale * (text_embeds @ image_embeds.T)  # (B, B)
    logits_per_image = logits_per_text.T  # (B, B)
    
    # Labels: diagonal (each text matches its corresponding image)
    batch_size = image_embeds.size(0)
    labels = torch.arange(batch_size, device=image_embeds.device)
    
    # Cross-entropy losses
    loss_t = F.cross_entropy(logits_per_text, labels)
    loss_i = F.cross_entropy(logits_per_image, labels)
    
    # Average
    loss = (loss_t + loss_i) / 2.0
    
    return loss

# Setup optimizer (only trainable parameters)
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=TRAINING_CONFIG["learning_rate"],
    weight_decay=0.01
)

# Setup learning rate scheduler (linear warmup + cosine decay)
# Calculate steps per epoch accounting for gradient accumulation
# Defensive check: ensure we have batches to train on
if len(train_dataloader) == 0:
    raise RuntimeError("train_dataloader has 0 batches. Check dataset filtering / loading.")

steps_per_epoch = int(np.ceil(len(train_dataloader) / TRAINING_CONFIG["gradient_accumulation_steps"]))
steps_per_epoch = max(1, steps_per_epoch)  # Ensure at least 1 step
total_steps = steps_per_epoch * TRAINING_CONFIG["num_epochs"]
# Ensure warmup_steps doesn't exceed total_steps
warmup_steps = min(TRAINING_CONFIG["warmup_steps"], total_steps)

def get_lr_scheduler(optimizer, num_warmup_steps, num_training_steps):
    """Create learning rate scheduler with warmup."""
    from torch.optim.lr_scheduler import LambdaLR
    
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + np.cos(np.pi * progress)))
    
    return LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, warmup_steps, total_steps)

print(f"✓ Training setup complete")
print(f"  Optimizer: AdamW (lr={TRAINING_CONFIG['learning_rate']})")
print(f"  Scheduler: Linear warmup + Cosine decay")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")


## 8. Training Loop


In [ ]:
# Training logs
training_logs = {
    "run_id": RUN_ID,
    "start_time": datetime.utcnow().isoformat() + "Z",
    "config": {
        "model": MODEL_NAME,
        "lora": LORA_CONFIG,
        "training": TRAINING_CONFIG,
    },
    "epochs": [],
    "total_time_seconds": 0,
    "peak_memory_mb": 0,
}

# Get device for inputs
if DEVICE == "cuda":
    main_device = next(model.parameters()).device
else:
    main_device = torch.device("cpu")

print("=" * 60)
print("Starting Training")
print("=" * 60)
print(f"Device: {main_device}")
print(f"Total epochs: {TRAINING_CONFIG['num_epochs']}")
batches_per_epoch = len(train_dataloader)
optimizer_steps_per_epoch = int(np.ceil(batches_per_epoch / TRAINING_CONFIG["gradient_accumulation_steps"]))
print(f"Batches per epoch: {batches_per_epoch}")
print(f"Optimizer steps per epoch: {optimizer_steps_per_epoch}")
print(f"Total optimizer steps: {total_steps}")
print()

model.train()
training_start_time = time.time()
global_step = 0
peak_memory = 0
step_losses = []  # Track losses per step (not per batch)

# Setup mixed precision training (AMP) for GPU
# Use generic torch.amp API (works correctly on both CPU and CUDA)
use_amp = (DEVICE == "cuda")
if use_amp:
    scaler = torch.amp.GradScaler("cuda", enabled=True)
    print("✓ Mixed precision (AMP) enabled for GPU training")
else:
    scaler = None  # No scaler needed for CPU
    print("ℹ Mixed precision disabled (CPU training)")

# Set random seeds for reproducibility
torch.manual_seed(TRAINING_CONFIG["random_seed"])
np.random.seed(TRAINING_CONFIG["random_seed"])
random.seed(TRAINING_CONFIG["random_seed"])

for epoch in range(TRAINING_CONFIG["num_epochs"]):
    epoch_start_time = time.time()
    epoch_losses = []
    running_loss = 0.0  # Accumulate loss across gradient accumulation steps
    
    print(f"\nEpoch {epoch + 1}/{TRAINING_CONFIG['num_epochs']}")
    print("-" * 60)
    
    # Reset failed image counter for this epoch
    train_dataset.failed_image_count = 0
    
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")
    
    # Track last batch_idx explicitly (more robust than using locals())
    last_batch_idx = -1
    for batch_idx, inputs in enumerate(progress_bar):
        # Skip batches with no valid samples (filtered out in collate_fn)
        if inputs is None:
            continue
        
        last_batch_idx = batch_idx
        # Move inputs to device
        input_ids = inputs["input_ids"].to(main_device)
        attention_mask = inputs.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(main_device)
        pixel_values = inputs["pixel_values"].to(main_device)
        
        # Forward pass using specific methods (avoids PEFT wrapper issues)
        # Use get_image_features and get_text_features instead of forward()
        text_inputs = {
            "input_ids": input_ids,
        }
        if attention_mask is not None:
            text_inputs["attention_mask"] = attention_mask
        
        image_inputs = {
            "pixel_values": pixel_values,
        }
        
        # Forward pass with mixed precision (AMP) when on GPU
        # Using enabled=use_amp for robustness (works even if refactored)
        with torch.amp.autocast("cuda", enabled=use_amp):
            # Get embeddings separately
            text_embeds = model.get_text_features(**text_inputs)
            image_embeds = model.get_image_features(**image_inputs)
            
            # Compute loss
            loss = clip_contrastive_loss(
                image_embeds,
                text_embeds,
                model.logit_scale
            )
            
            # Scale loss for gradient accumulation
            loss = loss / TRAINING_CONFIG["gradient_accumulation_steps"]
        
        # Backward pass (with scaler for AMP, direct for CPU)
        if use_amp:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        
        # Accumulate unscaled loss for this accumulation step
        running_loss += float(loss.item() * TRAINING_CONFIG["gradient_accumulation_steps"])
        
        # Store unscaled loss for epoch average
        epoch_losses.append(float(loss.item() * TRAINING_CONFIG["gradient_accumulation_steps"]))
        
        # Update weights (only after accumulation steps)
        if (batch_idx + 1) % TRAINING_CONFIG["gradient_accumulation_steps"] == 0:
            if use_amp:
                # Gradient clipping (especially important with trainable logit_scale)
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(
                    [p for p in model.parameters() if p.requires_grad],
                    max_norm=1.0
                )
                scaler.step(optimizer)
                scaler.update()
            else:
                # Gradient clipping for CPU
                torch.nn.utils.clip_grad_norm_(
                    [p for p in model.parameters() if p.requires_grad],
                    max_norm=1.0
                )
                optimizer.step()
            
            # Clamp logit_scale to prevent temperature explosion (CLIP best practice)
            with torch.no_grad():
                model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)
            
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            # Store average loss for this step (across accumulation)
            step_losses.append(running_loss / TRAINING_CONFIG["gradient_accumulation_steps"])
            running_loss = 0.0
        
        # Logging (by step, not by batch)
        if global_step > 0 and global_step % TRAINING_CONFIG["logging_steps"] == 0:
            current_lr = scheduler.get_last_lr()[0]
            # Use step_losses for accurate logging
            avg_loss = np.mean(step_losses[-TRAINING_CONFIG["logging_steps"]:]) if len(step_losses) >= TRAINING_CONFIG["logging_steps"] else np.mean(step_losses)
            # Log logit_scale (temperature) to monitor stability
            logit_scale_value = float(model.logit_scale.exp().item())
            progress_bar.set_postfix({
                "loss": f"{avg_loss:.4f}",
                "lr": f"{current_lr:.2e}",
                "temp": f"{logit_scale_value:.2f}",  # Temperature (exp of logit_scale)
                "step": global_step
            })
            
            # Store in training logs for analysis
            if "step_logs" not in training_logs:
                training_logs["step_logs"] = []
            training_logs["step_logs"].append({
                "global_step": global_step,
                "loss": float(avg_loss),
                "learning_rate": float(current_lr),
                "logit_scale": float(model.logit_scale.item()),
                "temperature": logit_scale_value
            })
        
        # Track peak memory
        if DEVICE == "cuda":
            current_memory = torch.cuda.max_memory_allocated() / 1024**2  # MB
            peak_memory = max(peak_memory, current_memory)
    
    # Process remaining gradients at end of epoch if not aligned with accumulation steps
    # Protection: ensure last_batch_idx is valid (should be >= 0 if dataloader has batches)
    if last_batch_idx >= 0:
        remaining_steps = (last_batch_idx + 1) % TRAINING_CONFIG["gradient_accumulation_steps"]
    else:
        remaining_steps = 0  # No batches processed
    
    if remaining_steps > 0:
        if use_amp:
            # Gradient clipping (especially important with trainable logit_scale)
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad],
                max_norm=1.0
            )
            scaler.step(optimizer)
            scaler.update()
        else:
            # Gradient clipping for CPU
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad],
                max_norm=1.0
            )
            optimizer.step()
        
        # Clamp logit_scale to prevent temperature explosion (CLIP best practice)
        with torch.no_grad():
            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)
        
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1
        # Store average loss for remaining accumulation steps
        step_losses.append(running_loss / remaining_steps)
        running_loss = 0.0
    
    epoch_time = time.time() - epoch_start_time
    avg_epoch_loss = np.mean(epoch_losses)
    
    # Calculate optimizer steps for this epoch
    batches = len(train_dataloader)
    optimizer_steps_this_epoch = int(np.ceil(batches / TRAINING_CONFIG["gradient_accumulation_steps"]))
    
    # Report failed image loads (guardrail: detect if training is contaminated)
    failed_images_this_epoch = train_dataset.failed_image_count
    total_samples = len(train_dataloader) * TRAINING_CONFIG["batch_size"]
    failure_rate = (failed_images_this_epoch / total_samples * 100) if total_samples > 0 else 0
    
    if failed_images_this_epoch > 0:
        print(f"  ⚠ Failed image loads: {failed_images_this_epoch} ({failure_rate:.2f}% of samples)")
        
        # Abort if contamination is too high (>0.5% threshold)
        CONTAMINATION_THRESHOLD = 0.5
        if failure_rate > CONTAMINATION_THRESHOLD:
            raise RuntimeError(
                f"Training aborted: image load failure rate ({failure_rate:.2f}%) exceeds threshold ({CONTAMINATION_THRESHOLD}%). "
                f"Training would be contaminated. Check image paths and MinIO connectivity."
            )
    
    epoch_log = {
        "epoch": epoch + 1,
        "avg_loss": float(avg_epoch_loss),
        "time_seconds": float(epoch_time),
        "batches": batches,
        "optimizer_steps": optimizer_steps_this_epoch,
        "failed_image_loads": failed_images_this_epoch,  # Track for analysis
    }
    training_logs["epochs"].append(epoch_log)
    
    print(f"  Average loss: {avg_epoch_loss:.4f}")
    print(f"  Time: {epoch_time:.2f} seconds")

training_time = time.time() - training_start_time
training_logs["total_time_seconds"] = float(training_time)
training_logs["peak_memory_mb"] = float(peak_memory)
training_logs["end_time"] = datetime.utcnow().isoformat() + "Z"
training_logs["total_steps"] = global_step

print("\n" + "=" * 60)
print("Training Complete")
print("=" * 60)
print(f"Total time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"Peak memory: {peak_memory:.2f} MB")
print(f"Total steps: {global_step}")


In [ ]:
def save_json_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as JSON to MinIO."""
    try:
        json_bytes = json.dumps(data, indent=2).encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=json_bytes,
            ContentType="application/json",
        )
        size_kb = len(json_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

def save_yaml_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as YAML to MinIO."""
    try:
        yaml_str = yaml.dump(data, default_flow_style=False, sort_keys=False)
        yaml_bytes = yaml_str.encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=yaml_bytes,
            ContentType="text/yaml",
        )
        size_kb = len(yaml_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

# Save adapters locally first, then upload to MinIO
print("Saving adapters...")
local_adapters_dir = Path(f"adapters_{RUN_ID}")
model.save_pretrained(str(local_adapters_dir))

# Upload adapters to MinIO (upload config and weights - supports both .bin and .safetensors)
print("Uploading adapters to MinIO...")
adapter_files = ["adapter_config.json", "adapter_model.bin", "adapter_model.safetensors"]
for file_name in adapter_files:
    file_path = local_adapters_dir / file_name
    if file_path.exists():
        with open(file_path, "rb") as f:
            s3.put_object(
                Bucket=FINE_TUNING_BUCKET,
                Key=f"{ADAPTERS_DIR}/{file_name}",
                Body=f.read(),
            )
        print(f"  ✓ Uploaded {file_name}")

# Save training logs
print("\nSaving training logs...")
save_json_to_minio(training_logs, FINE_TUNING_BUCKET, LOGS_KEY)

# Save configuration
config_data = {
    "run_id": RUN_ID,
    "model": MODEL_NAME,
    "method": "lora",
    "created_at": datetime.utcnow().isoformat() + "Z",
    "lora_config": LORA_CONFIG,
    "training_config": TRAINING_CONFIG,
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
    },
}
save_yaml_to_minio(config_data, FINE_TUNING_BUCKET, CONFIG_KEY)

print(f"\n✅ Adapters and logs saved successfully!")
print(f"  Adapters: s3://{FINE_TUNING_BUCKET}/{ADAPTERS_DIR}/")
print(f"  Logs: s3://{FINE_TUNING_BUCKET}/{LOGS_KEY}")
print(f"  Config: s3://{FINE_TUNING_BUCKET}/{CONFIG_KEY}")


## 10. Evaluation (Same Protocol as Baseline)

Now we evaluate the fine-tuned model using the same evaluation functions as the baseline.


In [ ]:
# Set model to evaluation mode
model.eval()

# Generate embeddings for test set (same as baseline)
print("=" * 60)
print("Generating Embeddings for Evaluation")
print("=" * 60)

def generate_image_embeddings(
    df: pd.DataFrame, 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> Tuple[np.ndarray, List[str]]:
    """Generate image embeddings for all images in the dataset."""
    model.eval()
    embeddings = []
    image_keys = []
    failed_keys = []
    
    unique_images = df["image_key"].unique()
    print(f"Generating embeddings for {len(unique_images)} unique images...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(unique_images), batch_size)):
            batch_keys = unique_images[i:i+batch_size]
            batch_images = []
            valid_keys = []
            
            for img_key in batch_keys:
                bucket, key = get_image_path_in_minio(img_key)
                img = load_image_from_minio(bucket, key)
                if img is not None:
                    batch_images.append(img)
                    valid_keys.append(img_key)
                else:
                    failed_keys.append(img_key)
            
            if not batch_images:
                continue
            
            inputs = processor(images=batch_images, return_tensors="pt", padding=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_image_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            
            embeddings.append(batch_emb.cpu().numpy())
            image_keys.extend(valid_keys)
    
    if failed_keys:
        print(f"⚠ Failed to load {len(failed_keys)} images")
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(image_keys)} image embeddings (shape: {embeddings.shape})")
    return embeddings, image_keys

def generate_text_embeddings(
    texts: List[str], 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> np.ndarray:
    """Generate text embeddings for all captions."""
    model.eval()
    embeddings = []
    
    print(f"Generating embeddings for {len(texts)} texts...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]
            inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_text_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            embeddings.append(batch_emb.cpu().numpy())
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(texts)} text embeddings (shape: {embeddings.shape})")
    return embeddings

start_time = time.time()

# Generate embeddings
image_embeddings, image_keys = generate_image_embeddings(
    test_df, model, processor, main_device, batch_size=16  # Smaller batch for LoRA
)

unique_captions = test_df["caption"].unique().tolist()
text_embeddings = generate_text_embeddings(unique_captions, model, processor, main_device, batch_size=16)

# Create official copy to ensure alignment with text_embeddings (never modify this)
unique_captions_list = list(unique_captions)  # Explicit copy to prevent accidental modification

embedding_time = time.time() - start_time
print(f"\n✓ Embedding generation complete in {embedding_time:.2f} seconds")

# Create mapping dictionaries
image_key_to_idx = {key: idx for idx, key in enumerate(image_keys)}
caption_to_idx = {caption: idx for idx, caption in enumerate(unique_captions_list)}

# Precompute lookup dictionaries for O(1) access (same as baseline)
print("\nPrecomputing lookup dictionaries...")

# recipe_id -> list of image indices
recipe_to_image_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_images = test_df[test_df["recipe_id"] == recipe_id]["image_key"].unique()
    recipe_to_image_indices[recipe_id_str] = [
        image_key_to_idx.get(img_key) 
        for img_key in recipe_images 
        if image_key_to_idx.get(img_key) is not None
    ]

# recipe_id -> list of caption indices (one caption per recipe for text→image query)
# Use longest caption as representative (most informative)
recipe_to_caption_indices: Dict[str, List[int]] = {}
recipe_to_caption_text: Dict[str, str] = {}
# recipe_id -> list of ALL caption indices (for image→text ground truth)
recipe_to_all_caption_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_captions = test_df[test_df["recipe_id"] == recipe_id]["caption"].unique()
    if len(recipe_captions) == 0:
        continue
    
    # Use longest caption as representative (most informative)
    # If multiple have same length, use first after sorting alphabetically (deterministic)
    representative_caption = max(recipe_captions, key=lambda c: (len(c), c))
    
    # Store representative caption for text→image queries
    cap_idx = caption_to_idx.get(representative_caption)
    if cap_idx is not None:
        recipe_to_caption_indices[recipe_id_str] = [cap_idx]
        recipe_to_caption_text[recipe_id_str] = representative_caption
    
    # Store ALL captions for image→text ground truth (more fair if multiple captions per recipe)
    all_caption_indices = []
    for caption in recipe_captions:
        cap_idx = caption_to_idx.get(caption)
        if cap_idx is not None:
            all_caption_indices.append(cap_idx)
    recipe_to_all_caption_indices[recipe_id_str] = sorted(set(all_caption_indices))  # Remove duplicates, sorted for determinism

# image_key -> recipe_id (as string for consistency)
image_key_to_recipe_id: Dict[str, str] = {}
for _, row in test_df.iterrows():
    if row["image_key"] not in image_key_to_recipe_id:
        image_key_to_recipe_id[row["image_key"]] = str(row["recipe_id"])

print(f"✓ Precomputed lookups for {len(recipe_to_image_indices)} recipes")


## 11. Evaluation Functions (Same as Baseline)


In [ ]:
# Copy evaluation functions from baseline (same protocol by recipe_id)
def compute_recall_at_k(scores: np.ndarray, ground_truth_indices: List[int], k: int) -> float:
    """
    Compute Recall@K.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
        k: top-K to consider
    
    Returns:
        recall@k: 1.0 if any ground truth is in top-K, else 0.0
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    top_k_indices = np.argsort(scores)[-k:][::-1]  # Top K, highest first
    top_k_set = set(top_k_indices)
    
    # Check if any ground truth is in top-K
    for gt_idx in ground_truth_indices:
        if gt_idx in top_k_set:
            return 1.0
    
    return 0.0

def compute_mrr(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute Mean Reciprocal Rank.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        mrr: 1/rank of first correct result, or 0.0 if none found
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return 1.0 / rank
    
    return 0.0

def compute_first_hit_rank(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute rank of first correct result (first hit rank).
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        first_hit_rank: rank of first correct result (1-indexed), or len(scores)+1 if not found
    """
    if len(ground_truth_indices) == 0:
        return float(len(scores) + 1)
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return float(rank)
    
    return float(len(scores) + 1)  # Not found

def evaluate_text_to_image_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_indices: Dict[str, List[int]],
    recipe_to_image_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Text → Image retrieval by recipe_id.
    
    For each recipe_id, use its representative caption to retrieve images.
    Ground truth: all images from that recipe_id.
    """
    print("\nEvaluating Text → Image retrieval (by recipe_id)...")
    
    unique_recipe_ids = test_df["recipe_id"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for recipe_id in tqdm(unique_recipe_ids, desc="Processing recipes"):
        recipe_id_str = str(recipe_id)
        # Get representative caption embedding (first caption for this recipe)
        caption_indices = recipe_to_caption_indices.get(recipe_id_str, [])
        if not caption_indices:
            continue
        
        # Use first caption (or could average multiple)
        text_idx = caption_indices[0]
        text_emb = text_embeddings[text_idx]
        
        # Compute similarities with all images
        similarities = image_embeddings @ text_emb  # (n_images,)
        
        # Get ground truth: all images from this recipe_id
        ground_truth_indices = recipe_to_image_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

def evaluate_image_to_text_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Image → Text retrieval by recipe_id.
    
    For each image, retrieve captions. Ground truth: ALL captions from the same recipe_id.
    """
    print("\nEvaluating Image → Text retrieval (by recipe_id)...")
    
    unique_images = test_df["image_key"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for img_key in tqdm(unique_images, desc="Processing images"):
        # Get image embedding
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        
        # Compute similarities with all captions
        similarities = text_embeddings @ img_emb  # (n_texts,)
        
        # Get recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: ALL captions from the same recipe_id (more fair)
        ground_truth_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

# Run evaluation
print("=" * 60)
print("Running Evaluation")
print("=" * 60)

eval_start_time = time.time()

# Text → Image retrieval (by recipe_id)
text_to_image_results = evaluate_text_to_image_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_indices,
    recipe_to_image_indices,
    image_key_to_idx,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

# Image → Text retrieval (by recipe_id)
image_to_text_results = evaluate_image_to_text_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    image_key_to_idx,
    image_key_to_recipe_id,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

eval_time = time.time() - eval_start_time

print("\n" + "=" * 60)
print("Evaluation Results")
print("=" * 60)

# Mapping for display names (only for specific metrics)
METRIC_DISPLAY_NAMES = {
    "MedianRank_first_hit": "MedianRank",
    "MeanRank_first_hit": "MeanRank"
}

print("\n📊 Text → Image Retrieval:")
for metric, value in text_to_image_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print("\n📊 Image → Text Retrieval:")
for metric, value in image_to_text_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print(f"\n⏱ Evaluation time: {eval_time:.2f} seconds")


## 11.5. Visualize Retrieval Results

Visualize the retrieval results to see how well the fine-tuned model performs:
- **Text → Image**: For each recipe, show the top-5 retrieved images
- **Image → Text**: For each image, show the top-5 retrieved captions


In [ ]:
# Visualize results
print("=" * 60)
print("Visualizing Retrieval Results")
print("=" * 60)

# Verify required variables exist (safety check)
assert "image_key_to_recipe_id" in globals(), "image_key_to_recipe_id not found. Run evaluation section first."
assert "text_embeddings" in globals(), "text_embeddings not found. Run evaluation section first."
assert "image_embeddings" in globals(), "image_embeddings not found. Run evaluation section first."
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."
assert "image_keys" in globals(), "image_keys not found. Run evaluation section first."

print("\n📝 Text → Image Retrieval (Top-5 images for each recipe):")
print("-" * 60)
visualize_text_to_image_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=3
)

print("\n🖼️ Image → Text Retrieval (Top-5 captions for each image):")
print("-" * 60)
visualize_image_to_text_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions_list,
    n_examples=3
)


## 12. Generate Qualitative Examples


In [ ]:
def generate_top5_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_text: Dict[str, str],
    recipe_to_image_indices: Dict[str, List[int]],
    caption_to_idx: Dict[str, int],
    image_keys: List[str],
    image_key_to_recipe_id: Dict[str, str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 retrievals by recipe_id.
    """
    
    examples = []
    
    # Filter recipes with valid ground truth before sampling (avoid bias)
    valid_recipe_ids = []
    for recipe_id in test_df["recipe_id"].unique():
        recipe_id_str = str(recipe_id)
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        gt_image_indices = recipe_to_image_indices.get(recipe_id_str, [])
        if len(gt_image_indices) > 0:  # Only include if has valid GT
            valid_recipe_ids.append(recipe_id)
    
    # Sample random recipes from valid ones
    if len(valid_recipe_ids) == 0:
        print("⚠ No valid recipes with ground truth images found")
        return examples
    
    sample_recipe_ids = pd.Series(valid_recipe_ids).sample(
        min(n_examples, len(valid_recipe_ids)),
        random_state=RANDOM_SEED
    )
    
    for recipe_id in sample_recipe_ids:
        recipe_id_str = str(recipe_id)
        # Get representative caption for this recipe
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        
        # Get caption embedding
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        
        text_emb = text_embeddings[text_idx]
        similarities = image_embeddings @ text_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: images from this recipe_id
        gt_image_indices = set(recipe_to_image_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved images
        top5_images = []
        for i, idx in enumerate(top5_indices):
            img_key = image_keys[idx]
            img_recipe_id = image_key_to_recipe_id.get(img_key)
            is_correct = idx in gt_image_indices
            
            top5_images.append({
                "rank": i + 1,
                "image_key": img_key,
                "recipe_id": img_recipe_id,
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "text_to_image",
            "recipe_id": recipe_id_str,
            "query_caption": caption,
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_images
        })
    
    return examples

def generate_image_to_text_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    recipe_to_caption_text: Dict[str, str],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    unique_captions: List[str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 captions for images (Image→Text retrieval).
    """
    examples = []
    
    # Filter images with valid ground truth before sampling (avoid bias)
    valid_images = []
    for img_key in test_df["image_key"].unique():
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        gt_caption_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        if len(gt_caption_indices) > 0:  # Only include if has valid GT
            valid_images.append(img_key)
    
    # Sample random images from valid ones
    if len(valid_images) == 0:
        print("⚠ No valid images with ground truth captions found")
        return examples
    
    sample_images = pd.Series(valid_images).sample(
        min(n_examples, len(valid_images)),
        random_state=RANDOM_SEED
    )
    
    for img_key in sample_images:
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        similarities = text_embeddings @ img_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: all captions from this recipe_id
        gt_caption_indices = set(recipe_to_all_caption_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved captions
        top5_captions = []
        for i, idx in enumerate(top5_indices):
            caption = unique_captions[idx]
            is_correct = idx in gt_caption_indices
            
            top5_captions.append({
                "rank": i + 1,
                "caption": caption,
                "caption_index": int(idx),
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "image_to_text",
            "recipe_id": recipe_id_str,
            "query_image_key": img_key,
            "recipe_representative_caption": recipe_to_caption_text.get(recipe_id_str),
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_captions
        })
    
    return examples

# Generate examples
N_QUALITATIVE_EXAMPLES = 5
print("Generating qualitative examples...")

# Verify unique_captions_list exists (should be from evaluation section)
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."

# Text → Image examples
text_to_image_examples = generate_top5_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=N_QUALITATIVE_EXAMPLES
)

# Image → Text examples
image_to_text_examples = generate_image_to_text_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions=unique_captions_list,
    n_examples=N_QUALITATIVE_EXAMPLES
)

print(f"✓ Generated {len(text_to_image_examples)} text→image examples (requested: {N_QUALITATIVE_EXAMPLES})")
print(f"✓ Generated {len(image_to_text_examples)} image→text examples (requested: {N_QUALITATIVE_EXAMPLES})")


## 13. Save Results


In [ ]:
# Create results dictionary
results = {
    "metadata": {
        "run_id": RUN_ID,
        "model": MODEL_NAME,
        "method": "lora",
        "created_at": datetime.utcnow().isoformat() + "Z",
        "device": str(main_device),
        "train_dataset": TRAIN_AUGMENTED_KEY,
        "test_dataset": TEST_MANIFEST_KEY,
        "filter_applied": "label == 1 (positive pairs only) for training",
    },
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
        "frozen": False,
    },
    "training_info": {
        "total_time_seconds": training_logs["total_time_seconds"],
        "peak_memory_mb": training_logs["peak_memory_mb"],
        "total_steps": training_logs["total_steps"],
        "num_epochs": TRAINING_CONFIG["num_epochs"],
        "final_loss": float(training_logs["epochs"][-1]["avg_loss"]) if training_logs["epochs"] else None,
    },
    "dataset_info": {
        "test_pairs": len(test_df),
        "test_recipes": test_df["recipe_id"].nunique(),
        "test_images": test_df["image_key"].nunique(),
        "test_captions": test_df["caption"].nunique(),
    },
    "metrics": {
        "text_to_image": text_to_image_results,
        "image_to_text": image_to_text_results,
    },
    "timing": {
        "embedding_generation_seconds": embedding_time,
        "evaluation_seconds": eval_time,
        "total_seconds": embedding_time + eval_time,
    },
}

# Save results
print("=" * 60)
print("Saving Results to MinIO")
print("=" * 60)

save_json_to_minio(results, FINE_TUNING_BUCKET, RESULTS_KEY)
save_json_to_minio({
    "text_to_image_examples": text_to_image_examples,
    "image_to_text_examples": image_to_text_examples
}, FINE_TUNING_BUCKET, EXAMPLES_KEY)

print(f"\n✅ Results saved successfully!")
print(f"  Results: s3://{FINE_TUNING_BUCKET}/{RESULTS_KEY}")
print(f"  Examples: s3://{FINE_TUNING_BUCKET}/{EXAMPLES_KEY}")

# Print summary
print("\n" + "=" * 60)
print("Summary")
print("=" * 60)
print(f"Run ID: {RUN_ID}")
print(f"Model: {MODEL_NAME} (LoRA fine-tuned)")
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Peak memory: {peak_memory:.2f} MB")
print(f"\nText → Image:")
print(f"  R@1: {text_to_image_results.get('R@1', 0):.4f}")
print(f"  R@5: {text_to_image_results.get('R@5', 0):.4f}")
print(f"  R@10: {text_to_image_results.get('R@10', 0):.4f}")
if "MRR" in text_to_image_results:
    print(f"  MRR: {text_to_image_results['MRR']:.4f}")
print(f"\nImage → Text:")
print(f"  R@1: {image_to_text_results.get('R@1', 0):.4f}")
print(f"  R@5: {image_to_text_results.get('R@5', 0):.4f}")
print(f"  R@10: {image_to_text_results.get('R@10', 0):.4f}")
if "MRR" in image_to_text_results:
    print(f"  MRR: {image_to_text_results['MRR']:.4f}")
print(f"\n📁 All outputs saved to: s3://{FINE_TUNING_BUCKET}/{RUN_DIR}/")


In [ ]:
def save_json_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as JSON to MinIO."""
    try:
        json_bytes = json.dumps(data, indent=2).encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=json_bytes,
            ContentType="application/json",
        )
        size_kb = len(json_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

def save_yaml_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as YAML to MinIO."""
    try:
        yaml_str = yaml.dump(data, default_flow_style=False, sort_keys=False)
        yaml_bytes = yaml_str.encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=yaml_bytes,
            ContentType="text/yaml",
        )
        size_kb = len(yaml_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

# Save adapters locally first, then upload to MinIO
print("Saving adapters...")
local_adapters_dir = Path(f"adapters_{RUN_ID}")
model.save_pretrained(str(local_adapters_dir))

# Upload adapters to MinIO (upload config and weights - supports both .bin and .safetensors)
print("Uploading adapters to MinIO...")
adapter_files = ["adapter_config.json", "adapter_model.bin", "adapter_model.safetensors"]
for file_name in adapter_files:
    file_path = local_adapters_dir / file_name
    if file_path.exists():
        with open(file_path, "rb") as f:
            s3.put_object(
                Bucket=FINE_TUNING_BUCKET,
                Key=f"{ADAPTERS_DIR}/{file_name}",
                Body=f.read(),
            )
        print(f"  ✓ Uploaded {file_name}")

# Save training logs
print("\nSaving training logs...")
save_json_to_minio(training_logs, FINE_TUNING_BUCKET, LOGS_KEY)

# Save configuration
config_data = {
    "run_id": RUN_ID,
    "model": MODEL_NAME,
    "method": "lora",
    "created_at": datetime.utcnow().isoformat() + "Z",
    "lora_config": LORA_CONFIG,
    "training_config": TRAINING_CONFIG,
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
    },
}
save_yaml_to_minio(config_data, FINE_TUNING_BUCKET, CONFIG_KEY)

print(f"\n✅ Adapters and logs saved successfully!")
print(f"  Adapters: s3://{FINE_TUNING_BUCKET}/{ADAPTERS_DIR}/")
print(f"  Logs: s3://{FINE_TUNING_BUCKET}/{LOGS_KEY}")
print(f"  Config: s3://{FINE_TUNING_BUCKET}/{CONFIG_KEY}")


## 10. Evaluation (Same Protocol as Baseline)

Now we evaluate the fine-tuned model using the same evaluation functions as the baseline.


In [ ]:
# Set model to evaluation mode
model.eval()

# Generate embeddings for test set (same as baseline)
print("=" * 60)
print("Generating Embeddings for Evaluation")
print("=" * 60)

def generate_image_embeddings(
    df: pd.DataFrame, 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> Tuple[np.ndarray, List[str]]:
    """Generate image embeddings for all images in the dataset."""
    model.eval()
    embeddings = []
    image_keys = []
    failed_keys = []
    
    unique_images = df["image_key"].unique()
    print(f"Generating embeddings for {len(unique_images)} unique images...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(unique_images), batch_size)):
            batch_keys = unique_images[i:i+batch_size]
            batch_images = []
            valid_keys = []
            
            for img_key in batch_keys:
                bucket, key = get_image_path_in_minio(img_key)
                img = load_image_from_minio(bucket, key)
                if img is not None:
                    batch_images.append(img)
                    valid_keys.append(img_key)
                else:
                    failed_keys.append(img_key)
            
            if not batch_images:
                continue
            
            inputs = processor(images=batch_images, return_tensors="pt", padding=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_image_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            
            embeddings.append(batch_emb.cpu().numpy())
            image_keys.extend(valid_keys)
    
    if failed_keys:
        print(f"⚠ Failed to load {len(failed_keys)} images")
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(image_keys)} image embeddings (shape: {embeddings.shape})")
    return embeddings, image_keys

def generate_text_embeddings(
    texts: List[str], 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> np.ndarray:
    """Generate text embeddings for all captions."""
    model.eval()
    embeddings = []
    
    print(f"Generating embeddings for {len(texts)} texts...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]
            inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_text_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            embeddings.append(batch_emb.cpu().numpy())
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(texts)} text embeddings (shape: {embeddings.shape})")
    return embeddings

start_time = time.time()

# Generate embeddings
image_embeddings, image_keys = generate_image_embeddings(
    test_df, model, processor, main_device, batch_size=16  # Smaller batch for LoRA
)

unique_captions = test_df["caption"].unique().tolist()
text_embeddings = generate_text_embeddings(unique_captions, model, processor, main_device, batch_size=16)

# Create official copy to ensure alignment with text_embeddings (never modify this)
unique_captions_list = list(unique_captions)  # Explicit copy to prevent accidental modification

embedding_time = time.time() - start_time
print(f"\n✓ Embedding generation complete in {embedding_time:.2f} seconds")

# Create mapping dictionaries
image_key_to_idx = {key: idx for idx, key in enumerate(image_keys)}
caption_to_idx = {caption: idx for idx, caption in enumerate(unique_captions_list)}

# Precompute lookup dictionaries for O(1) access (same as baseline)
print("\nPrecomputing lookup dictionaries...")

# recipe_id -> list of image indices
recipe_to_image_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_images = test_df[test_df["recipe_id"] == recipe_id]["image_key"].unique()
    recipe_to_image_indices[recipe_id_str] = [
        image_key_to_idx.get(img_key) 
        for img_key in recipe_images 
        if image_key_to_idx.get(img_key) is not None
    ]

# recipe_id -> list of caption indices (one caption per recipe for text→image query)
# Use longest caption as representative (most informative)
recipe_to_caption_indices: Dict[str, List[int]] = {}
recipe_to_caption_text: Dict[str, str] = {}
# recipe_id -> list of ALL caption indices (for image→text ground truth)
recipe_to_all_caption_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_captions = test_df[test_df["recipe_id"] == recipe_id]["caption"].unique()
    if len(recipe_captions) == 0:
        continue
    
    # Use longest caption as representative (most informative)
    # If multiple have same length, use first after sorting alphabetically (deterministic)
    representative_caption = max(recipe_captions, key=lambda c: (len(c), c))
    
    # Store representative caption for text→image queries
    cap_idx = caption_to_idx.get(representative_caption)
    if cap_idx is not None:
        recipe_to_caption_indices[recipe_id_str] = [cap_idx]
        recipe_to_caption_text[recipe_id_str] = representative_caption
    
    # Store ALL captions for image→text ground truth (more fair if multiple captions per recipe)
    all_caption_indices = []
    for caption in recipe_captions:
        cap_idx = caption_to_idx.get(caption)
        if cap_idx is not None:
            all_caption_indices.append(cap_idx)
    recipe_to_all_caption_indices[recipe_id_str] = sorted(set(all_caption_indices))  # Remove duplicates, sorted for determinism

# image_key -> recipe_id (as string for consistency)
image_key_to_recipe_id: Dict[str, str] = {}
for _, row in test_df.iterrows():
    if row["image_key"] not in image_key_to_recipe_id:
        image_key_to_recipe_id[row["image_key"]] = str(row["recipe_id"])

print(f"✓ Precomputed lookups for {len(recipe_to_image_indices)} recipes")


## 11. Evaluation Functions (Same as Baseline)


In [ ]:
# Copy evaluation functions from baseline (same protocol by recipe_id)
def compute_recall_at_k(scores: np.ndarray, ground_truth_indices: List[int], k: int) -> float:
    """
    Compute Recall@K.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
        k: top-K to consider
    
    Returns:
        recall@k: 1.0 if any ground truth is in top-K, else 0.0
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    top_k_indices = np.argsort(scores)[-k:][::-1]  # Top K, highest first
    top_k_set = set(top_k_indices)
    
    # Check if any ground truth is in top-K
    for gt_idx in ground_truth_indices:
        if gt_idx in top_k_set:
            return 1.0
    
    return 0.0

def compute_mrr(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute Mean Reciprocal Rank.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        mrr: 1/rank of first correct result, or 0.0 if none found
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return 1.0 / rank
    
    return 0.0

def compute_first_hit_rank(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute rank of first correct result (first hit rank).
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        first_hit_rank: rank of first correct result (1-indexed), or len(scores)+1 if not found
    """
    if len(ground_truth_indices) == 0:
        return float(len(scores) + 1)
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return float(rank)
    
    return float(len(scores) + 1)  # Not found

def evaluate_text_to_image_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_indices: Dict[str, List[int]],
    recipe_to_image_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Text → Image retrieval by recipe_id.
    
    For each recipe_id, use its representative caption to retrieve images.
    Ground truth: all images from that recipe_id.
    """
    print("\nEvaluating Text → Image retrieval (by recipe_id)...")
    
    unique_recipe_ids = test_df["recipe_id"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for recipe_id in tqdm(unique_recipe_ids, desc="Processing recipes"):
        recipe_id_str = str(recipe_id)
        # Get representative caption embedding (first caption for this recipe)
        caption_indices = recipe_to_caption_indices.get(recipe_id_str, [])
        if not caption_indices:
            continue
        
        # Use first caption (or could average multiple)
        text_idx = caption_indices[0]
        text_emb = text_embeddings[text_idx]
        
        # Compute similarities with all images
        similarities = image_embeddings @ text_emb  # (n_images,)
        
        # Get ground truth: all images from this recipe_id
        ground_truth_indices = recipe_to_image_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

def evaluate_image_to_text_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Image → Text retrieval by recipe_id.
    
    For each image, retrieve captions. Ground truth: ALL captions from the same recipe_id.
    """
    print("\nEvaluating Image → Text retrieval (by recipe_id)...")
    
    unique_images = test_df["image_key"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for img_key in tqdm(unique_images, desc="Processing images"):
        # Get image embedding
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        
        # Compute similarities with all captions
        similarities = text_embeddings @ img_emb  # (n_texts,)
        
        # Get recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: ALL captions from the same recipe_id (more fair)
        ground_truth_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

# Run evaluation
print("=" * 60)
print("Running Evaluation")
print("=" * 60)

eval_start_time = time.time()

# Text → Image retrieval (by recipe_id)
text_to_image_results = evaluate_text_to_image_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_indices,
    recipe_to_image_indices,
    image_key_to_idx,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

# Image → Text retrieval (by recipe_id)
image_to_text_results = evaluate_image_to_text_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    image_key_to_idx,
    image_key_to_recipe_id,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

eval_time = time.time() - eval_start_time

print("\n" + "=" * 60)
print("Evaluation Results")
print("=" * 60)

# Mapping for display names (only for specific metrics)
METRIC_DISPLAY_NAMES = {
    "MedianRank_first_hit": "MedianRank",
    "MeanRank_first_hit": "MeanRank"
}

print("\n📊 Text → Image Retrieval:")
for metric, value in text_to_image_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print("\n📊 Image → Text Retrieval:")
for metric, value in image_to_text_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print(f"\n⏱ Evaluation time: {eval_time:.2f} seconds")


## 11.5. Visualize Retrieval Results

Visualize the retrieval results to see how well the fine-tuned model performs:
- **Text → Image**: For each recipe, show the top-5 retrieved images
- **Image → Text**: For each image, show the top-5 retrieved captions


In [ ]:
# Visualize results
print("=" * 60)
print("Visualizing Retrieval Results")
print("=" * 60)

# Verify required variables exist (safety check)
assert "image_key_to_recipe_id" in globals(), "image_key_to_recipe_id not found. Run evaluation section first."
assert "text_embeddings" in globals(), "text_embeddings not found. Run evaluation section first."
assert "image_embeddings" in globals(), "image_embeddings not found. Run evaluation section first."
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."
assert "image_keys" in globals(), "image_keys not found. Run evaluation section first."

print("\n📝 Text → Image Retrieval (Top-5 images for each recipe):")
print("-" * 60)
visualize_text_to_image_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=3
)

print("\n🖼️ Image → Text Retrieval (Top-5 captions for each image):")
print("-" * 60)
visualize_image_to_text_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions_list,
    n_examples=3
)


## 12. Generate Qualitative Examples


In [ ]:
def generate_top5_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_text: Dict[str, str],
    recipe_to_image_indices: Dict[str, List[int]],
    caption_to_idx: Dict[str, int],
    image_keys: List[str],
    image_key_to_recipe_id: Dict[str, str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 retrievals by recipe_id.
    """
    
    examples = []
    
    # Filter recipes with valid ground truth before sampling (avoid bias)
    valid_recipe_ids = []
    for recipe_id in test_df["recipe_id"].unique():
        recipe_id_str = str(recipe_id)
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        gt_image_indices = recipe_to_image_indices.get(recipe_id_str, [])
        if len(gt_image_indices) > 0:  # Only include if has valid GT
            valid_recipe_ids.append(recipe_id)
    
    # Sample random recipes from valid ones
    if len(valid_recipe_ids) == 0:
        print("⚠ No valid recipes with ground truth images found")
        return examples
    
    sample_recipe_ids = pd.Series(valid_recipe_ids).sample(
        min(n_examples, len(valid_recipe_ids)),
        random_state=RANDOM_SEED
    )
    
    for recipe_id in sample_recipe_ids:
        recipe_id_str = str(recipe_id)
        # Get representative caption for this recipe
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        
        # Get caption embedding
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        
        text_emb = text_embeddings[text_idx]
        similarities = image_embeddings @ text_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: images from this recipe_id
        gt_image_indices = set(recipe_to_image_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved images
        top5_images = []
        for i, idx in enumerate(top5_indices):
            img_key = image_keys[idx]
            img_recipe_id = image_key_to_recipe_id.get(img_key)
            is_correct = idx in gt_image_indices
            
            top5_images.append({
                "rank": i + 1,
                "image_key": img_key,
                "recipe_id": img_recipe_id,
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "text_to_image",
            "recipe_id": recipe_id_str,
            "query_caption": caption,
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_images
        })
    
    return examples

def generate_image_to_text_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    recipe_to_caption_text: Dict[str, str],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    unique_captions: List[str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 captions for images (Image→Text retrieval).
    """
    examples = []
    
    # Filter images with valid ground truth before sampling (avoid bias)
    valid_images = []
    for img_key in test_df["image_key"].unique():
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        gt_caption_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        if len(gt_caption_indices) > 0:  # Only include if has valid GT
            valid_images.append(img_key)
    
    # Sample random images from valid ones
    if len(valid_images) == 0:
        print("⚠ No valid images with ground truth captions found")
        return examples
    
    sample_images = pd.Series(valid_images).sample(
        min(n_examples, len(valid_images)),
        random_state=RANDOM_SEED
    )
    
    for img_key in sample_images:
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        similarities = text_embeddings @ img_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: all captions from this recipe_id
        gt_caption_indices = set(recipe_to_all_caption_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved captions
        top5_captions = []
        for i, idx in enumerate(top5_indices):
            caption = unique_captions[idx]
            is_correct = idx in gt_caption_indices
            
            top5_captions.append({
                "rank": i + 1,
                "caption": caption,
                "caption_index": int(idx),
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "image_to_text",
            "recipe_id": recipe_id_str,
            "query_image_key": img_key,
            "recipe_representative_caption": recipe_to_caption_text.get(recipe_id_str),
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_captions
        })
    
    return examples

# Generate examples
N_QUALITATIVE_EXAMPLES = 5
print("Generating qualitative examples...")

# Verify unique_captions_list exists (should be from evaluation section)
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."

# Text → Image examples
text_to_image_examples = generate_top5_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=N_QUALITATIVE_EXAMPLES
)

# Image → Text examples
image_to_text_examples = generate_image_to_text_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions=unique_captions_list,
    n_examples=N_QUALITATIVE_EXAMPLES
)

print(f"✓ Generated {len(text_to_image_examples)} text→image examples (requested: {N_QUALITATIVE_EXAMPLES})")
print(f"✓ Generated {len(image_to_text_examples)} image→text examples (requested: {N_QUALITATIVE_EXAMPLES})")


## 13. Save Results


In [ ]:
# Create results dictionary
results = {
    "metadata": {
        "run_id": RUN_ID,
        "model": MODEL_NAME,
        "method": "lora",
        "created_at": datetime.utcnow().isoformat() + "Z",
        "device": str(main_device),
        "train_dataset": TRAIN_AUGMENTED_KEY,
        "test_dataset": TEST_MANIFEST_KEY,
        "filter_applied": "label == 1 (positive pairs only) for training",
    },
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
        "frozen": False,
    },
    "training_info": {
        "total_time_seconds": training_logs["total_time_seconds"],
        "peak_memory_mb": training_logs["peak_memory_mb"],
        "total_steps": training_logs["total_steps"],
        "num_epochs": TRAINING_CONFIG["num_epochs"],
        "final_loss": float(training_logs["epochs"][-1]["avg_loss"]) if training_logs["epochs"] else None,
    },
    "dataset_info": {
        "test_pairs": len(test_df),
        "test_recipes": test_df["recipe_id"].nunique(),
        "test_images": test_df["image_key"].nunique(),
        "test_captions": test_df["caption"].nunique(),
    },
    "metrics": {
        "text_to_image": text_to_image_results,
        "image_to_text": image_to_text_results,
    },
    "timing": {
        "embedding_generation_seconds": embedding_time,
        "evaluation_seconds": eval_time,
        "total_seconds": embedding_time + eval_time,
    },
}

# Save results
print("=" * 60)
print("Saving Results to MinIO")
print("=" * 60)

save_json_to_minio(results, FINE_TUNING_BUCKET, RESULTS_KEY)
save_json_to_minio({
    "text_to_image_examples": text_to_image_examples,
    "image_to_text_examples": image_to_text_examples
}, FINE_TUNING_BUCKET, EXAMPLES_KEY)

print(f"\n✅ Results saved successfully!")
print(f"  Results: s3://{FINE_TUNING_BUCKET}/{RESULTS_KEY}")
print(f"  Examples: s3://{FINE_TUNING_BUCKET}/{EXAMPLES_KEY}")

# Print summary
print("\n" + "=" * 60)
print("Summary")
print("=" * 60)
print(f"Run ID: {RUN_ID}")
print(f"Model: {MODEL_NAME} (LoRA fine-tuned)")
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Peak memory: {peak_memory:.2f} MB")
print(f"\nText → Image:")
print(f"  R@1: {text_to_image_results.get('R@1', 0):.4f}")
print(f"  R@5: {text_to_image_results.get('R@5', 0):.4f}")
print(f"  R@10: {text_to_image_results.get('R@10', 0):.4f}")
if "MRR" in text_to_image_results:
    print(f"  MRR: {text_to_image_results['MRR']:.4f}")
print(f"\nImage → Text:")
print(f"  R@1: {image_to_text_results.get('R@1', 0):.4f}")
print(f"  R@5: {image_to_text_results.get('R@5', 0):.4f}")
print(f"  R@10: {image_to_text_results.get('R@10', 0):.4f}")
if "MRR" in image_to_text_results:
    print(f"  MRR: {image_to_text_results['MRR']:.4f}")
print(f"\n📁 All outputs saved to: s3://{FINE_TUNING_BUCKET}/{RUN_DIR}/")


## 2. Initialize MinIO Client


In [ ]:
# Initialize S3/MinIO client
session = boto3.session.Session(
    aws_access_key_id=MINIO_USER,
    aws_secret_access_key=MINIO_PASSWORD,
    region_name="us-east-1"
)
s3 = session.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    config=Config(signature_version="s3v4", s3={"addressing_style": "path"})
)

def ensure_bucket_exists(bucket: str) -> bool:
    """Create bucket if it doesn't exist."""
    try:
        s3.head_bucket(Bucket=bucket)
        return True
    except ClientError as e:
        error_code = e.response.get("Error", {}).get("Code", "")
        if error_code in ("404", "NoSuchBucket"):
            try:
                s3.create_bucket(Bucket=bucket)
                print(f"✓ Created bucket '{bucket}'")
                return True
            except ClientError as create_error:
                print(f"✗ Failed to create bucket '{bucket}': {create_error}")
                return False
        return False

# Verify buckets
print("Checking buckets...")
ensure_bucket_exists(FINE_TUNING_BUCKET)
print("✓ Buckets ready")


## 3. Load Training and Test Datasets


In [ ]:
def load_csv_from_minio(bucket: str, key: str) -> pd.DataFrame:
    """Load CSV file from MinIO into a DataFrame."""
    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(io.BytesIO(obj["Body"].read()))
        print(f"✓ Loaded {len(df)} rows from s3://{bucket}/{key}")
        return df
    except ClientError as e:
        print(f"✗ Failed to load s3://{bucket}/{key}: {e}")
        return pd.DataFrame()

# Load training dataset (augmented, with negatives)
print("Loading augmented training dataset...")
train_full_df = load_csv_from_minio(FINE_TUNING_BUCKET, TRAIN_AUGMENTED_KEY)

if train_full_df.empty:
    raise RuntimeError(f"Could not load training data from s3://{FINE_TUNING_BUCKET}/{TRAIN_AUGMENTED_KEY}")

print(f"\nFull training dataset shape: {train_full_df.shape}")
print(f"Columns: {list(train_full_df.columns)}")
if "label" in train_full_df.columns:
    print(f"\nLabel distribution:")
    print(train_full_df["label"].value_counts())
    
    # Filter only positive pairs (label=1)
    train_df = train_full_df[train_full_df["label"] == 1].copy()
    print(f"\n✓ Filtered to {len(train_df)} positive pairs (label=1)")
    print(f"  Removed {len(train_full_df) - len(train_df)} negative pairs")
else:
    # If no label column, assume all are positive
    train_df = train_full_df.copy()
    print(f"\n✓ No label column found, using all {len(train_df)} pairs as positive")

# Load test dataset (from baseline split, already filtered)
print("\nLoading test dataset...")
test_df = load_csv_from_minio(FINE_TUNING_BUCKET, TEST_MANIFEST_KEY)

if test_df.empty:
    raise RuntimeError(f"Could not load test data from s3://{FINE_TUNING_BUCKET}/{TEST_MANIFEST_KEY}")

print(f"\nTraining dataset:")
print(f"  Pairs: {len(train_df)}")
print(f"  Recipes: {train_df['recipe_id'].nunique()}")
print(f"  Images: {train_df['image_key'].nunique()}")
print(f"  Captions: {train_df['caption'].nunique()}")

print(f"\nTest dataset:")
print(f"  Pairs: {len(test_df)}")
print(f"  Recipes: {test_df['recipe_id'].nunique()}")
print(f"  Images: {test_df['image_key'].nunique()}")
print(f"  Captions: {test_df['caption'].nunique()}")


## 4. Helper Functions for Image Loading


In [ ]:
def load_image_from_minio(bucket: str, key: str) -> Optional[Image.Image]:
    """Load an image from MinIO."""
    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        img = Image.open(io.BytesIO(obj["Body"].read()))
        img.load()
        return img.convert("RGB")
    except Exception as e:
        return None

# Cache for image paths to avoid head_object per sample (performance optimization)
_image_path_cache: Dict[str, Tuple[str, str]] = {}

def build_image_path_cache(df: pd.DataFrame) -> Dict[str, Tuple[str, str]]:
    """
    Pre-build cache of image_key -> (bucket, full_key) to avoid head_object calls during training.
    This significantly improves throughput when loading images from MinIO.
    """
    print("Building image path cache...")
    cache = {}
    bucket = FINE_TUNING_BUCKET
    unique_keys = df["image_key"].unique()
    
    for image_key in tqdm(unique_keys, desc="Caching image paths"):
        # Check if it's already a full path
        if image_key.startswith("images/") or image_key.startswith("augmented_images/"):
            cache[image_key] = (bucket, image_key)
        else:
            # Try images/ first
            key = f"{IMAGES_PREFIX}/{image_key}"
            # Check if exists, if not try augmented_images/
            try:
                s3.head_object(Bucket=bucket, Key=key)
                cache[image_key] = (bucket, key)
            except ClientError as e:
                if e.response.get("Error", {}).get("Code") == "404":
                    # Try augmented_images/
                    key = f"augmented_images/{image_key}"
                    cache[image_key] = (bucket, key)
                else:
                    raise
    
    print(f"✓ Cached {len(cache)} image paths")
    return cache

def get_image_path_in_minio(image_key: str) -> Tuple[str, str]:
    """
    Determine bucket and full key for an image (uses cache if available).
    
    Images can be in:
    - fine-tuning-zone/images/...
    - fine-tuning-zone/augmented_images/...
    
    If cache is built, uses it. Otherwise falls back to head_object (slower).
    """
    # Use cache if available
    if image_key in _image_path_cache:
        return _image_path_cache[image_key]
    
    # Fallback: check if it's already a full path
    bucket = FINE_TUNING_BUCKET
    if image_key.startswith("images/") or image_key.startswith("augmented_images/"):
        key = image_key
    else:
        # Try images/ first
        key = f"{IMAGES_PREFIX}/{image_key}"
        # Check if exists, if not try augmented_images/
        try:
            s3.head_object(Bucket=bucket, Key=key)
        except ClientError as e:
            if e.response.get("Error", {}).get("Code") == "404":
                # Try augmented_images/
                key = f"augmented_images/{image_key}"
            else:
                raise
    
    return bucket, key


## 5. Load CLIP Model with LoRA


In [ ]:
if not TRANSFORMERS_AVAILABLE or not PEFT_AVAILABLE:
    raise ImportError("Required libraries not available. Install: pip install transformers peft")

print(f"Loading CLIP model with LoRA: {MODEL_NAME}")
print(f"Device: {DEVICE}")

# Load model
print("Loading model...")
model = CLIPModel.from_pretrained(MODEL_NAME).to(DEVICE)

# Load processor
processor = CLIPProcessor.from_pretrained(MODEL_NAME)

print("Model ready for training (LoRA)")

# Validate target_modules match actual CLIP module names (guardrail)
print("\nValidating target_modules against actual CLIP architecture...")
all_module_names = [name for name, _ in model.named_modules()]
target_modules = LORA_CONFIG["target_modules"]
matched_modules = []
for target in target_modules:
    matches = [name for name in all_module_names if target in name]
    if matches:
        matched_modules.extend(matches)
        print(f"  ✓ '{target}' matches: {matches[:3]}...")  # Show first 3
    else:
        print(f"  ⚠ '{target}' not found in model modules!")

if not matched_modules:
    print("\n⚠ WARNING: No target_modules matched! LoRA may not be applied.")
    print("Available attention-related modules (sample):")
    attn_modules = [name for name in all_module_names if "attn" in name.lower() or "proj" in name.lower()]
    for mod in attn_modules[:10]:  # Show first 10
        print(f"  - {mod}")
    print("  ... (use these to adjust target_modules)")

# Configure LoRA
lora_config = LoraConfig(
    r=LORA_CONFIG["r"],
    lora_alpha=LORA_CONFIG["lora_alpha"],
    lora_dropout=LORA_CONFIG["lora_dropout"],
    bias=LORA_CONFIG["bias"],
    task_type=LORA_CONFIG["task_type"],
    target_modules=LORA_CONFIG["target_modules"],
)

# Apply LoRA adapters
print("Applying LoRA adapters...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
print("\n" + "=" * 60)
print("Model Information")
print("=" * 60)
model.print_trainable_parameters()

# Check logit_scale trainability
print(f"\nlogit_scale requires_grad: {model.logit_scale.requires_grad}")
if not model.logit_scale.requires_grad:
    print("⚠ logit_scale is frozen. Enabling it for better fine-tuning (1 extra parameter, almost free).")
    model.logit_scale.requires_grad_(True)
    print("✓ logit_scale is now trainable")

# Get model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✓ Model loaded successfully")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Trainable ratio: {trainable_params/total_params*100:.2f}%")


## 6. Create DataLoader for Training


In [ ]:
from torch.utils.data import Dataset, DataLoader
import random

class CLIPDataset(Dataset):
    """Dataset for CLIP training with image-text pairs."""
    
    def __init__(self, df: pd.DataFrame, processor, s3_client, bucket: str):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.s3 = s3_client
        self.bucket = bucket
        self.failed_image_count = 0
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image
        bucket, key = get_image_path_in_minio(row["image_key"])
        img = load_image_from_minio(bucket, key)
        
        # Get caption
        caption = row["caption"]
        
        if img is None:
            # Return a placeholder if image fails to load
            self.failed_image_count += 1
            return {"image": None, "text": caption, "is_valid": False}
        
        return {
            "image": img,
            "text": caption, 
            "is_valid": True
        }

def collate_fn(batch):
    """Collate function for DataLoader."""
    # Filter out invalid samples (images that failed to load)
    batch = [b for b in batch if b.get("is_valid", True) and b["image"] is not None]
    
    if len(batch) == 0:
        return None  # Signal to skip this batch
    
    images = [item["image"] for item in batch]
    texts = [item["text"] for item in batch]
    
    # Process with CLIP processor
    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    
    # Filter to only include what CLIPModel.forward() accepts
    # CLIPModel expects: input_ids, attention_mask, pixel_values
    allowed_keys = {"input_ids", "attention_mask", "pixel_values"}
    return {k: v for k, v in inputs.items() if k in allowed_keys}

# Build image path cache (performance optimization: avoid head_object per sample)
print("Building image path cache for training set...")
_image_path_cache.update(build_image_path_cache(train_df))

# Create datasets and dataloaders
print("\nCreating datasets and dataloaders...")

train_dataset = CLIPDataset(train_df, processor, s3, FINE_TUNING_BUCKET)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=TRAINING_CONFIG["batch_size"],
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,  # Set to 0 to avoid issues with MinIO
)

print(f"✓ Created training dataloader")
print(f"  Total batches: {len(train_dataloader)}")
print(f"  Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"  Gradient accumulation steps: {TRAINING_CONFIG['gradient_accumulation_steps']}")
print(f"  Effective batch size: {TRAINING_CONFIG['batch_size'] * TRAINING_CONFIG['gradient_accumulation_steps']}")



## 11.5. Visualize Retrieval Results

Visualize the retrieval results to see how well the fine-tuned model performs:
- **Text → Image**: For each caption, show the top-5 retrieved images
- **Image → Text**: For each image, show the top-5 retrieved captions


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Use random seed from training config
RANDOM_SEED = TRAINING_CONFIG["random_seed"]

def visualize_text_to_image_retrieval(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_text: Dict[str, str],
    recipe_to_image_indices: Dict[str, List[int]],
    caption_to_idx: Dict[str, int],
    image_keys: List[str],
    image_key_to_recipe_id: Dict[str, str],
    n_examples: int = 3
):
    """
    Visualize Text → Image retrieval: for each recipe, show top-5 images.
    """
    
    # Sample random recipes
    unique_recipe_ids = test_df["recipe_id"].unique()
    sample_recipe_ids = pd.Series(unique_recipe_ids).sample(
        min(n_examples, len(unique_recipe_ids)),
        random_state=RANDOM_SEED
    )
    
    for recipe_idx, recipe_id in enumerate(sample_recipe_ids):
        recipe_id_str = str(recipe_id)
        # Get representative caption for this recipe
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        
        # Get caption embedding
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        
        text_emb = text_embeddings[text_idx]
        similarities = image_embeddings @ text_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]  # Top 5, highest first
        
        # Get ground truth: images from this recipe_id
        gt_image_indices = set(recipe_to_image_indices.get(recipe_id_str, []))
        
        # Create figure with more vertical space for captions
        fig = plt.figure(figsize=(16, 5))
        gs = gridspec.GridSpec(1, 6, width_ratios=[2, 1, 1, 1, 1, 1], hspace=0.3, bottom=0.15)
        
        # Show query caption
        ax0 = fig.add_subplot(gs[0])
        ax0.text(0.5, 0.5, f'Query:\n"{caption}"', 
                ha='center', va='center', fontsize=14, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
        ax0.axis('off')
        
        # Show top-5 images
        for i, img_idx in enumerate(top5_indices):
            img_key = image_keys[img_idx]
            bucket, key = get_image_path_in_minio(img_key)
            img = load_image_from_minio(bucket, key)
            
            if img is None:
                continue
            
            # Check if this image is from the ground truth recipe
            is_correct = img_idx in gt_image_indices
            
            # Get recipe_id for this image to show its representative caption
            img_recipe_id = image_key_to_recipe_id.get(img_key, "Unknown")
            # Note: Shows representative caption of the recipe (longest caption), not necessarily the exact caption for this image
            img_caption = recipe_to_caption_text.get(img_recipe_id, "Unknown")
            
            ax = fig.add_subplot(gs[i + 1])
            ax.imshow(img)
            ax.axis('off')
            
            # Color border: green if correct, red if incorrect
            border_color = 'green' if is_correct else 'red'
            border_width = 3 if is_correct else 2
            for spine in ax.spines.values():
                spine.set_edgecolor(border_color)
                spine.set_linewidth(border_width)
            
            # Title with similarity score
            similarity_score = similarities[img_idx]
            rank = i + 1
            title = f"Rank {rank}\n{similarity_score:.3f}"
            if is_correct:
                title += "\n✓ Correct"
            ax.set_title(title, fontsize=10, color=border_color, fontweight='bold')
            
            # Add recipe caption below the image
            # Truncate long captions for display
            display_caption = img_caption if len(img_caption) <= 40 else img_caption[:37] + "..."
            ax.text(0.5, -0.15, f'"{display_caption}"', 
                   ha='center', va='top', fontsize=9, 
                   transform=ax.transAxes,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'),
                   wrap=True)
        
        plt.suptitle(f'Text → Image Retrieval Example {recipe_idx + 1} (Recipe: {recipe_id_str})', 
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        print()

def visualize_image_to_text_retrieval(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    recipe_to_caption_text: Dict[str, str],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    unique_captions: List[str],
    n_examples: int = 3
):
    """
    Visualize Image → Text retrieval: for each image, show top-5 captions.
    """
    
    # Sample random images
    unique_images = test_df["image_key"].unique()
    sample_images = pd.Series(unique_images).sample(
        min(n_examples, len(unique_images)),
        random_state=RANDOM_SEED
    )
    
    for img_idx_example, img_key in enumerate(sample_images):
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        similarities = text_embeddings @ img_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]  # Top 5, highest first
        
        # Get ground truth: recipe_id for this image
        gt_recipe_id_str = image_key_to_recipe_id.get(img_key)
        # Use all captions for ground truth (consistent with evaluation)
        gt_caption_indices = set(recipe_to_all_caption_indices.get(gt_recipe_id_str, [])) if gt_recipe_id_str else set()
        
        # Load query image
        bucket, key = get_image_path_in_minio(img_key)
        query_img = load_image_from_minio(bucket, key)
        
        if query_img is None:
            continue
        
        # Create figure with more vertical space for recipe name
        fig = plt.figure(figsize=(16, 4))
        gs = gridspec.GridSpec(1, 6, width_ratios=[1, 1, 1, 1, 1, 1], hspace=0.3, bottom=0.15)
        
        # Show query image
        ax0 = fig.add_subplot(gs[0])
        ax0.imshow(query_img)
        ax0.axis('off')
        ax0.set_title('Query Image', fontsize=12, fontweight='bold')
        
        # Add recipe name below the query image
        if gt_recipe_id_str:
            recipe_caption = recipe_to_caption_text.get(gt_recipe_id_str, "Unknown")
            # Truncate long captions for display
            display_recipe = recipe_caption if len(recipe_caption) <= 40 else recipe_caption[:37] + "..."
            ax0.text(0.5, -0.15, f'Recipe: {gt_recipe_id_str}\n"{display_recipe}"', 
                   ha='center', va='top', fontsize=9, 
                   transform=ax0.transAxes,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'),
                   wrap=True)
        
        # Show top-5 captions
        for i, text_idx in enumerate(top5_indices):
            caption = unique_captions[text_idx]
            
            # Check if this caption is from the ground truth recipe
            is_correct = text_idx in gt_caption_indices
            
            ax = fig.add_subplot(gs[i + 1])
            ax.axis('off')
            
            # Color border: green if correct, red if incorrect
            border_color = 'green' if is_correct else 'red'
            border_width = 3 if is_correct else 2
            
            # Create text box
            similarity_score = similarities[text_idx]
            rank = i + 1
            text_content = f"Rank {rank}\n\n\"{caption}\"\n\nSimilarity: {similarity_score:.3f}"
            if is_correct:
                text_content += "\n\n✓ Correct"
            
            ax.text(0.5, 0.5, text_content,
                   ha='center', va='center', fontsize=10,
                   bbox=dict(boxstyle='round', facecolor='lightyellow', 
                            edgecolor=border_color, linewidth=border_width, alpha=0.8),
                   wrap=True)
        
        plt.suptitle(f'Image → Text Retrieval Example {img_idx_example + 1} (Recipe: {gt_recipe_id_str})', 
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        print()

# Visualization functions defined above
# They will be called after embeddings are generated (see section after evaluation)


# This section has been moved to after the evaluation (section 11.5)


In [23]:
# This cell will be moved to after the evaluation section
# The visualization functions are now in the correct location (after section 11)


In [ ]:
def clip_contrastive_loss(image_embeds, text_embeds, logit_scale):
    """
    CLIP contrastive loss with in-batch negatives.
    
    Args:
        image_embeds: Image embeddings (batch_size, embed_dim)
        text_embeds: Text embeddings (batch_size, embed_dim)
        logit_scale: Learnable temperature parameter
    
    Returns:
        loss: Contrastive loss
    """
    # Normalize embeddings
    image_embeds = F.normalize(image_embeds, dim=-1)
    text_embeds = F.normalize(text_embeds, dim=-1)
    
    # Compute logits
    logit_scale = logit_scale.exp()
    logits_per_text = logit_scale * (text_embeds @ image_embeds.T)  # (B, B)
    logits_per_image = logits_per_text.T  # (B, B)
    
    # Labels: diagonal (each text matches its corresponding image)
    batch_size = image_embeds.size(0)
    labels = torch.arange(batch_size, device=image_embeds.device)
    
    # Cross-entropy losses
    loss_t = F.cross_entropy(logits_per_text, labels)
    loss_i = F.cross_entropy(logits_per_image, labels)
    
    # Average
    loss = (loss_t + loss_i) / 2.0
    
    return loss

# Setup optimizer (only trainable parameters)
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=TRAINING_CONFIG["learning_rate"],
    weight_decay=0.01
)

# Setup learning rate scheduler (linear warmup + cosine decay)
# Calculate steps per epoch accounting for gradient accumulation
# Defensive check: ensure we have batches to train on
if len(train_dataloader) == 0:
    raise RuntimeError("train_dataloader has 0 batches. Check dataset filtering / loading.")

steps_per_epoch = int(np.ceil(len(train_dataloader) / TRAINING_CONFIG["gradient_accumulation_steps"]))
steps_per_epoch = max(1, steps_per_epoch)  # Ensure at least 1 step
total_steps = steps_per_epoch * TRAINING_CONFIG["num_epochs"]
# Ensure warmup_steps doesn't exceed total_steps
warmup_steps = min(TRAINING_CONFIG["warmup_steps"], total_steps)

def get_lr_scheduler(optimizer, num_warmup_steps, num_training_steps):
    """Create learning rate scheduler with warmup."""
    from torch.optim.lr_scheduler import LambdaLR
    
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + np.cos(np.pi * progress)))
    
    return LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, warmup_steps, total_steps)

print(f"✓ Training setup complete")
print(f"  Optimizer: AdamW (lr={TRAINING_CONFIG['learning_rate']})")
print(f"  Scheduler: Linear warmup + Cosine decay")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")


## 8. Training Loop


In [ ]:
# Training logs
training_logs = {
    "run_id": RUN_ID,
    "start_time": datetime.utcnow().isoformat() + "Z",
    "config": {
        "model": MODEL_NAME,
        "lora": LORA_CONFIG,
        "training": TRAINING_CONFIG,
    },
    "epochs": [],
    "total_time_seconds": 0,
    "peak_memory_mb": 0,
}

# Get device for inputs
if DEVICE == "cuda":
    main_device = next(model.parameters()).device
else:
    main_device = torch.device("cpu")

print("=" * 60)
print("Starting Training")
print("=" * 60)
print(f"Device: {main_device}")
print(f"Total epochs: {TRAINING_CONFIG['num_epochs']}")
batches_per_epoch = len(train_dataloader)
optimizer_steps_per_epoch = int(np.ceil(batches_per_epoch / TRAINING_CONFIG["gradient_accumulation_steps"]))
print(f"Batches per epoch: {batches_per_epoch}")
print(f"Optimizer steps per epoch: {optimizer_steps_per_epoch}")
print(f"Total optimizer steps: {total_steps}")
print()

model.train()
training_start_time = time.time()
global_step = 0
peak_memory = 0
step_losses = []  # Track losses per step (not per batch)

# Setup mixed precision training (AMP) for GPU
# Use generic torch.amp API (works correctly on both CPU and CUDA)
use_amp = (DEVICE == "cuda")
if use_amp:
    scaler = torch.amp.GradScaler("cuda", enabled=True)
    print("✓ Mixed precision (AMP) enabled for GPU training")
else:
    scaler = None  # No scaler needed for CPU
    print("ℹ Mixed precision disabled (CPU training)")

# Set random seeds for reproducibility
torch.manual_seed(TRAINING_CONFIG["random_seed"])
np.random.seed(TRAINING_CONFIG["random_seed"])
random.seed(TRAINING_CONFIG["random_seed"])

for epoch in range(TRAINING_CONFIG["num_epochs"]):
    epoch_start_time = time.time()
    epoch_losses = []
    running_loss = 0.0  # Accumulate loss across gradient accumulation steps
    
    print(f"\nEpoch {epoch + 1}/{TRAINING_CONFIG['num_epochs']}")
    print("-" * 60)
    
    # Reset failed image counter for this epoch
    train_dataset.failed_image_count = 0
    
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")
    
    # Track last batch_idx explicitly (more robust than using locals())
    last_batch_idx = -1
    for batch_idx, inputs in enumerate(progress_bar):
        # Skip batches with no valid samples (filtered out in collate_fn)
        if inputs is None:
            continue
        
        last_batch_idx = batch_idx
        # Move inputs to device
        input_ids = inputs["input_ids"].to(main_device)
        attention_mask = inputs.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(main_device)
        pixel_values = inputs["pixel_values"].to(main_device)
        
        # Forward pass using specific methods (avoids PEFT wrapper issues)
        # Use get_image_features and get_text_features instead of forward()
        text_inputs = {
            "input_ids": input_ids,
        }
        if attention_mask is not None:
            text_inputs["attention_mask"] = attention_mask
        
        image_inputs = {
            "pixel_values": pixel_values,
        }
        
        # Forward pass with mixed precision (AMP) when on GPU
        # Using enabled=use_amp for robustness (works even if refactored)
        with torch.amp.autocast("cuda", enabled=use_amp):
            # Get embeddings separately
            text_embeds = model.get_text_features(**text_inputs)
            image_embeds = model.get_image_features(**image_inputs)
            
            # Compute loss
            loss = clip_contrastive_loss(
                image_embeds,
                text_embeds,
                model.logit_scale
            )
            
            # Scale loss for gradient accumulation
            loss = loss / TRAINING_CONFIG["gradient_accumulation_steps"]
        
        # Backward pass (with scaler for AMP, direct for CPU)
        if use_amp:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        
        # Accumulate unscaled loss for this accumulation step
        running_loss += float(loss.item() * TRAINING_CONFIG["gradient_accumulation_steps"])
        
        # Store unscaled loss for epoch average
        epoch_losses.append(float(loss.item() * TRAINING_CONFIG["gradient_accumulation_steps"]))
        
        # Update weights (only after accumulation steps)
        if (batch_idx + 1) % TRAINING_CONFIG["gradient_accumulation_steps"] == 0:
            if use_amp:
                # Gradient clipping (especially important with trainable logit_scale)
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(
                    [p for p in model.parameters() if p.requires_grad],
                    max_norm=1.0
                )
                scaler.step(optimizer)
                scaler.update()
            else:
                # Gradient clipping for CPU
                torch.nn.utils.clip_grad_norm_(
                    [p for p in model.parameters() if p.requires_grad],
                    max_norm=1.0
                )
                optimizer.step()
            
            # Clamp logit_scale to prevent temperature explosion (CLIP best practice)
            with torch.no_grad():
                model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)
            
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            # Store average loss for this step (across accumulation)
            step_losses.append(running_loss / TRAINING_CONFIG["gradient_accumulation_steps"])
            running_loss = 0.0
        
        # Logging (by step, not by batch)
        if global_step > 0 and global_step % TRAINING_CONFIG["logging_steps"] == 0:
            current_lr = scheduler.get_last_lr()[0]
            # Use step_losses for accurate logging
            avg_loss = np.mean(step_losses[-TRAINING_CONFIG["logging_steps"]:]) if len(step_losses) >= TRAINING_CONFIG["logging_steps"] else np.mean(step_losses)
            # Log logit_scale (temperature) to monitor stability
            logit_scale_value = float(model.logit_scale.exp().item())
            progress_bar.set_postfix({
                "loss": f"{avg_loss:.4f}",
                "lr": f"{current_lr:.2e}",
                "temp": f"{logit_scale_value:.2f}",  # Temperature (exp of logit_scale)
                "step": global_step
            })
            
            # Store in training logs for analysis
            if "step_logs" not in training_logs:
                training_logs["step_logs"] = []
            training_logs["step_logs"].append({
                "global_step": global_step,
                "loss": float(avg_loss),
                "learning_rate": float(current_lr),
                "logit_scale": float(model.logit_scale.item()),
                "temperature": logit_scale_value
            })
        
        # Track peak memory
        if DEVICE == "cuda":
            current_memory = torch.cuda.max_memory_allocated() / 1024**2  # MB
            peak_memory = max(peak_memory, current_memory)
    
    # Process remaining gradients at end of epoch if not aligned with accumulation steps
    # Protection: ensure last_batch_idx is valid (should be >= 0 if dataloader has batches)
    if last_batch_idx >= 0:
        remaining_steps = (last_batch_idx + 1) % TRAINING_CONFIG["gradient_accumulation_steps"]
    else:
        remaining_steps = 0  # No batches processed
    
    if remaining_steps > 0:
        if use_amp:
            # Gradient clipping (especially important with trainable logit_scale)
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad],
                max_norm=1.0
            )
            scaler.step(optimizer)
            scaler.update()
        else:
            # Gradient clipping for CPU
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad],
                max_norm=1.0
            )
            optimizer.step()
        
        # Clamp logit_scale to prevent temperature explosion (CLIP best practice)
        with torch.no_grad():
            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)
        
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1
        # Store average loss for remaining accumulation steps
        step_losses.append(running_loss / remaining_steps)
        running_loss = 0.0
    
    epoch_time = time.time() - epoch_start_time
    avg_epoch_loss = np.mean(epoch_losses)
    
    # Calculate optimizer steps for this epoch
    batches = len(train_dataloader)
    optimizer_steps_this_epoch = int(np.ceil(batches / TRAINING_CONFIG["gradient_accumulation_steps"]))
    
    # Report failed image loads (guardrail: detect if training is contaminated)
    failed_images_this_epoch = train_dataset.failed_image_count
    total_samples = len(train_dataloader) * TRAINING_CONFIG["batch_size"]
    failure_rate = (failed_images_this_epoch / total_samples * 100) if total_samples > 0 else 0
    
    if failed_images_this_epoch > 0:
        print(f"  ⚠ Failed image loads: {failed_images_this_epoch} ({failure_rate:.2f}% of samples)")
        
        # Abort if contamination is too high (>0.5% threshold)
        CONTAMINATION_THRESHOLD = 0.5
        if failure_rate > CONTAMINATION_THRESHOLD:
            raise RuntimeError(
                f"Training aborted: image load failure rate ({failure_rate:.2f}%) exceeds threshold ({CONTAMINATION_THRESHOLD}%). "
                f"Training would be contaminated. Check image paths and MinIO connectivity."
            )
    
    epoch_log = {
        "epoch": epoch + 1,
        "avg_loss": float(avg_epoch_loss),
        "time_seconds": float(epoch_time),
        "batches": batches,
        "optimizer_steps": optimizer_steps_this_epoch,
        "failed_image_loads": failed_images_this_epoch,  # Track for analysis
    }
    training_logs["epochs"].append(epoch_log)
    
    print(f"  Average loss: {avg_epoch_loss:.4f}")
    print(f"  Time: {epoch_time:.2f} seconds")

training_time = time.time() - training_start_time
training_logs["total_time_seconds"] = float(training_time)
training_logs["peak_memory_mb"] = float(peak_memory)
training_logs["end_time"] = datetime.utcnow().isoformat() + "Z"
training_logs["total_steps"] = global_step

print("\n" + "=" * 60)
print("Training Complete")
print("=" * 60)
print(f"Total time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"Peak memory: {peak_memory:.2f} MB")
print(f"Total steps: {global_step}")


# LoRA Fine-Tuning for CLIP

This notebook fine-tunes CLIP using **LoRA** (Low-Rank Adaptation) on the food recipe dataset.

**Hypothesis H1:** The fine-tuned model improves text↔image alignment compared to the CLIP baseline, measured with retrieval metrics (R@K, MRR).

**This notebook (M1 LoRA):**
- Loads pre-trained CLIP model (no quantization)
- Applies LoRA adapters to trainable modules
- Fine-tunes on training set using CLIP contrastive loss
- Evaluates on test set with **same protocol as baseline** (by recipe_id)
- Saves adapters, logs, and results for comparison

**Inputs:**
- `fine-tuning-zone/datasets/train_pairs_augmented_with_negatives.csv` (for training)
- `fine-tuning-zone/datasets/test_pairs_positive.csv` (for evaluation)
- `fine-tuning-zone/images/` and `fine-tuning-zone/augmented_images/` (images)

**Outputs:**
- `fine-tuning-zone/experiments/lora/run_{run_id}/` — Complete run directory with:
  - `config.yaml` — Training configuration
  - `adapters/` — LoRA adapter weights
  - `training_logs.json` — Loss and timing logs
  - `results_lora.json` — Evaluation metrics
  - `examples_top5.json` — Qualitative examples


## 1. Setup and Configuration


In [ ]:
import os
import io
import json
import time
import sys
import math
import torch
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union
from datetime import datetime

# Check Python environment
print("=" * 60)
print("Python Environment Check")
print("=" * 60)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we're in a virtual environment
venv_path = Path(sys.executable).parent.parent
if (venv_path / "pyvenv.cfg").exists() or "venv" in str(sys.executable).lower():
    print(f"✓ Running in virtual environment: {venv_path}")
else:
    print(f"⚠ Not in a virtual environment")
    print(f"  Current Python: {sys.executable}")
    print(f"  If you have a venv, make sure Jupyter is using it!")

import numpy as np
import pandas as pd
import torch.nn.functional as F
from PIL import Image
import boto3
from botocore.config import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv
from tqdm import tqdm
import yaml

# CLIP and LoRA imports
TRANSFORMERS_AVAILABLE = False
PEFT_AVAILABLE = False

try:
    from transformers import CLIPModel, CLIPProcessor
    TRANSFORMERS_AVAILABLE = True
    print("✓ transformers imported successfully")
except ImportError as e:
    print(f"✗ Failed to import transformers: {e}")

try:
    from peft import LoraConfig, get_peft_model
    PEFT_AVAILABLE = True
    print("✓ peft imported successfully")
except ImportError as e:
    print(f"✗ Failed to import peft: {e}")

print("\n" + "=" * 60)
print("Library Status")
print("=" * 60)
if not TRANSFORMERS_AVAILABLE:
    print("  ✗ transformers: NOT AVAILABLE")
    print("    Install with: pip install transformers")
else:
    print("  ✓ transformers: AVAILABLE")
    
if not PEFT_AVAILABLE:
    print("  ✗ peft: NOT AVAILABLE")
    print("    Install with: pip install peft")
else:
    print("  ✓ peft: AVAILABLE")

if TRANSFORMERS_AVAILABLE and PEFT_AVAILABLE:
    print("\n✓ All required libraries are available!")
    print("  You can proceed with training.")
elif not TRANSFORMERS_AVAILABLE or not PEFT_AVAILABLE:
    print("\n✗ Some required libraries are missing!")
    print("  Please install missing libraries before continuing.")

# Load environment variables
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent.parent

ENV_PATHS = [
    PROJECT_ROOT / "notebooks" / ".env",
    PROJECT_ROOT / "app" / ".env",
    PROJECT_ROOT / ".env",
]

env_loaded = False
for env_path in ENV_PATHS:
    if env_path.exists():
        load_dotenv(env_path)
        print(f"✓ Loaded .env from: {env_path}")
        env_loaded = True
        break

if not env_loaded:
    print("⚠ No .env file found, trying default load_dotenv()...")
    load_dotenv()

# MinIO Configuration
MINIO_USER = os.getenv("MINIO_USER")
MINIO_PASSWORD = os.getenv("MINIO_PASSWORD")
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT")

# Bucket configuration
FINE_TUNING_BUCKET = "fine-tuning-zone"
DATASETS_PREFIX = "datasets"
IMAGES_PREFIX = "images"
AUGMENTED_IMAGES_PREFIX = "augmented_images"
EXPERIMENTS_PREFIX = "experiments"

# Input/Output paths
TRAIN_AUGMENTED_KEY = f"{DATASETS_PREFIX}/train_pairs_augmented_with_negatives.csv"
TEST_MANIFEST_KEY = f"{DATASETS_PREFIX}/test_pairs_positive.csv"  # Test set from baseline split

# Model configuration
MODEL_NAME = "openai/clip-vit-base-patch32"  # Same as baseline

def pick_device():
    """Pick device, testing if CUDA actually works (not just available)."""
    if not torch.cuda.is_available():
        return "cpu"
    try:
        _ = torch.randn(1, device="cuda")
        return "cuda"
    except Exception as e:
        print(f"⚠ CUDA visible pero no usable: {e}")
        return "cpu"

DEVICE = pick_device()

# LoRA configuration
LORA_CONFIG = {
    "r": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "FEATURE_EXTRACTION",
    "target_modules": ["q_proj", "k_proj", "v_proj", "out_proj"],  # Only attention modules (safer for CLIP)
}

# Training configuration
TRAINING_CONFIG = {
    "learning_rate": 1e-4,
    "num_epochs": 3,
    "batch_size": 2,  # Smaller batch for CPU training
    "gradient_accumulation_steps": 16,  # Effective batch size = 2 * 16 = 32
    "warmup_steps": 100,
    "save_steps": 500,
    "logging_steps": 50,
    "random_seed": 42,
}

# Evaluation configuration (same as baseline)
K_VALUES = [1, 5, 10]
COMPUTE_MRR = True

# Generate run ID
RUN_ID = f"lora_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}"
RUN_DIR = f"{EXPERIMENTS_PREFIX}/lora/run_{RUN_ID}"

# Output paths
CONFIG_KEY = f"{RUN_DIR}/config.yaml"
ADAPTERS_DIR = f"{RUN_DIR}/adapters"
LOGS_KEY = f"{RUN_DIR}/training_logs.json"
RESULTS_KEY = f"{RUN_DIR}/results_lora.json"
EXAMPLES_KEY = f"{RUN_DIR}/examples_top5.json"

# Update config to include dataset info
TRAINING_CONFIG["train_dataset"] = TRAIN_AUGMENTED_KEY
TRAINING_CONFIG["test_dataset"] = TEST_MANIFEST_KEY
TRAINING_CONFIG["filter_applied"] = "label == 1 (positive pairs only)"

# Logit scale clamping constants (CLIP best practice)
LOGIT_SCALE_MIN = math.log(1/100)  # Minimum temperature: 0.01
LOGIT_SCALE_MAX = math.log(100)     # Maximum temperature: 100

print(f"Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Device: {DEVICE}")
print(f"  Method: LoRA")
print(f"  LoRA r: {LORA_CONFIG['r']}, alpha: {LORA_CONFIG['lora_alpha']}")
print(f"  Training: {TRAINING_CONFIG['num_epochs']} epochs, LR={TRAINING_CONFIG['learning_rate']}")
print(f"  Batch size: {TRAINING_CONFIG['batch_size']} (effective: {TRAINING_CONFIG['batch_size'] * TRAINING_CONFIG['gradient_accumulation_steps']})")
print(f"  Run ID: {RUN_ID}")


## 2. Initialize MinIO Client


In [ ]:
# Initialize S3/MinIO client
session = boto3.session.Session(
    aws_access_key_id=MINIO_USER,
    aws_secret_access_key=MINIO_PASSWORD,
    region_name="us-east-1"
)
s3 = session.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    config=Config(signature_version="s3v4", s3={"addressing_style": "path"})
)

def ensure_bucket_exists(bucket: str) -> bool:
    """Create bucket if it doesn't exist."""
    try:
        s3.head_bucket(Bucket=bucket)
        return True
    except ClientError as e:
        error_code = e.response.get("Error", {}).get("Code", "")
        if error_code in ("404", "NoSuchBucket"):
            try:
                s3.create_bucket(Bucket=bucket)
                print(f"✓ Created bucket '{bucket}'")
                return True
            except ClientError as create_error:
                print(f"✗ Failed to create bucket '{bucket}': {create_error}")
                return False
        return False

# Verify buckets
print("Checking buckets...")
ensure_bucket_exists(FINE_TUNING_BUCKET)
print("✓ Buckets ready")


## 3. Load Training and Test Datasets


In [ ]:
def load_csv_from_minio(bucket: str, key: str) -> pd.DataFrame:
    """Load CSV file from MinIO into a DataFrame."""
    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        df = pd.read_csv(io.BytesIO(obj["Body"].read()))
        print(f"✓ Loaded {len(df)} rows from s3://{bucket}/{key}")
        return df
    except ClientError as e:
        print(f"✗ Failed to load s3://{bucket}/{key}: {e}")
        return pd.DataFrame()

# Load training dataset (augmented, with negatives)
print("Loading augmented training dataset...")
train_full_df = load_csv_from_minio(FINE_TUNING_BUCKET, TRAIN_AUGMENTED_KEY)

if train_full_df.empty:
    raise RuntimeError(f"Could not load training data from s3://{FINE_TUNING_BUCKET}/{TRAIN_AUGMENTED_KEY}")

print(f"\nFull training dataset shape: {train_full_df.shape}")
print(f"Columns: {list(train_full_df.columns)}")
if "label" in train_full_df.columns:
    print(f"\nLabel distribution:")
    print(train_full_df["label"].value_counts())
    
    # Filter only positive pairs (label=1)
    train_df = train_full_df[train_full_df["label"] == 1].copy()
    print(f"\n✓ Filtered to {len(train_df)} positive pairs (label=1)")
    print(f"  Removed {len(train_full_df) - len(train_df)} negative pairs")
else:
    # If no label column, assume all are positive
    train_df = train_full_df.copy()
    print(f"\n✓ No label column found, using all {len(train_df)} pairs as positive")

# Load test dataset (from baseline split, already filtered)
print("\nLoading test dataset...")
test_df = load_csv_from_minio(FINE_TUNING_BUCKET, TEST_MANIFEST_KEY)

if test_df.empty:
    raise RuntimeError(f"Could not load test data from s3://{FINE_TUNING_BUCKET}/{TEST_MANIFEST_KEY}")

print(f"\nTraining dataset:")
print(f"  Pairs: {len(train_df)}")
print(f"  Recipes: {train_df['recipe_id'].nunique()}")
print(f"  Images: {train_df['image_key'].nunique()}")
print(f"  Captions: {train_df['caption'].nunique()}")

print(f"\nTest dataset:")
print(f"  Pairs: {len(test_df)}")
print(f"  Recipes: {test_df['recipe_id'].nunique()}")
print(f"  Images: {test_df['image_key'].nunique()}")
print(f"  Captions: {test_df['caption'].nunique()}")


## 4. Helper Functions for Image Loading


In [ ]:
def load_image_from_minio(bucket: str, key: str) -> Optional[Image.Image]:
    """Load an image from MinIO."""
    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        img = Image.open(io.BytesIO(obj["Body"].read()))
        img.load()
        return img.convert("RGB")
    except Exception as e:
        return None

def get_image_path_in_minio(image_key: str) -> Tuple[str, str]:
    """
    Determine bucket and full key for an image.
    
    Images can be in:
    - fine-tuning-zone/images/...
    - fine-tuning-zone/augmented_images/...
    
    Tries images/ first, then augmented_images/ if not found.
    """
    bucket = FINE_TUNING_BUCKET
    
    # Check if it's already a full path
    if image_key.startswith("images/") or image_key.startswith("augmented_images/"):
        key = image_key
    else:
        # Try images/ first
        key = f"{IMAGES_PREFIX}/{image_key}"
        # Check if exists, if not try augmented_images/
        try:
            s3.head_object(Bucket=bucket, Key=key)
        except ClientError as e:
            if e.response.get("Error", {}).get("Code") == "404":
                # Try augmented_images/
                key = f"augmented_images/{image_key}"
            else:
                raise
    
    return bucket, key


## 5. Load CLIP Model with LoRA


In [ ]:
if not TRANSFORMERS_AVAILABLE or not PEFT_AVAILABLE:
    raise ImportError("Required libraries not available. Install: pip install transformers peft")

print(f"Loading CLIP model with LoRA: {MODEL_NAME}")
print(f"Device: {DEVICE}")

# Load model
print("Loading model...")
model = CLIPModel.from_pretrained(MODEL_NAME).to(DEVICE)

# Load processor
processor = CLIPProcessor.from_pretrained(MODEL_NAME)

print("Model ready for training (LoRA)")

# Configure LoRA
lora_config = LoraConfig(
    r=LORA_CONFIG["r"],
    lora_alpha=LORA_CONFIG["lora_alpha"],
    lora_dropout=LORA_CONFIG["lora_dropout"],
    bias=LORA_CONFIG["bias"],
    task_type=LORA_CONFIG["task_type"],
    target_modules=LORA_CONFIG["target_modules"],
)

# Apply LoRA adapters
print("Applying LoRA adapters...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
print("\n" + "=" * 60)
print("Model Information")
print("=" * 60)
model.print_trainable_parameters()

# Check logit_scale trainability
print(f"\nlogit_scale requires_grad: {model.logit_scale.requires_grad}")
if not model.logit_scale.requires_grad:
    print("⚠ logit_scale is frozen. Enabling it for better fine-tuning (1 extra parameter, almost free).")
    model.logit_scale.requires_grad_(True)
    print("✓ logit_scale is now trainable")

# Get model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✓ Model loaded successfully")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Trainable ratio: {trainable_params/total_params*100:.2f}%")


## 6. Create DataLoader for Training


In [ ]:
from torch.utils.data import Dataset, DataLoader
import random

class CLIPDataset(Dataset):
    """Dataset for CLIP training with image-text pairs."""
    
    def __init__(self, df: pd.DataFrame, processor, s3_client, bucket: str):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.s3 = s3_client
        self.bucket = bucket
        self.failed_image_count = 0
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image
        bucket, key = get_image_path_in_minio(row["image_key"])
        img = load_image_from_minio(bucket, key)
        
        if img is None:
            # Return a placeholder if image fails to load
            self.failed_image_count += 1
            img = Image.new("RGB", (224, 224), color="black")
        
        # Get caption
        caption = row["caption"]
        
        return {
            "image": img,
            "text": caption,
        }

def collate_fn(batch):
    """Collate function for DataLoader."""
    images = [item["image"] for item in batch]
    texts = [item["text"] for item in batch]
    
    # Process with CLIP processor
    # Process text and images together (CLIPProcessor handles this correctly)
    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    
    # Filter to only include what CLIPModel.forward() accepts
    # CLIPModel expects: input_ids, attention_mask, pixel_values
    # Remove any other keys like inputs_embeds, etc.
    filtered_inputs = {}
    allowed_keys = {"input_ids", "attention_mask", "pixel_values"}
    
    for key, value in inputs.items():
        if key in allowed_keys:
            filtered_inputs[key] = value
    
    return filtered_inputs

# Create datasets and dataloaders
print("Creating datasets and dataloaders...")

train_dataset = CLIPDataset(train_df, processor, s3, FINE_TUNING_BUCKET)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=TRAINING_CONFIG["batch_size"],
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,  # Set to 0 to avoid issues with MinIO
)

print(f"✓ Created training dataloader")
print(f"  Total batches: {len(train_dataloader)}")
print(f"  Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"  Gradient accumulation steps: {TRAINING_CONFIG['gradient_accumulation_steps']}")
print(f"  Effective batch size: {TRAINING_CONFIG['batch_size'] * TRAINING_CONFIG['gradient_accumulation_steps']}")



## 9.5. Visualization Functions

Define functions to visualize retrieval results. These will be called after embeddings are generated (see section 11.5).


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Use random seed from training config
RANDOM_SEED = TRAINING_CONFIG["random_seed"]

def visualize_text_to_image_retrieval(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_text: Dict[str, str],
    recipe_to_image_indices: Dict[str, List[int]],
    caption_to_idx: Dict[str, int],
    image_keys: List[str],
    image_key_to_recipe_id: Dict[str, str],
    n_examples: int = 3
):
    """
    Visualize Text → Image retrieval: for each recipe, show top-5 images.
    """
    
    # Sample random recipes
    unique_recipe_ids = test_df["recipe_id"].unique()
    sample_recipe_ids = pd.Series(unique_recipe_ids).sample(
        min(n_examples, len(unique_recipe_ids)),
        random_state=RANDOM_SEED
    )
    
    for recipe_idx, recipe_id in enumerate(sample_recipe_ids):
        recipe_id_str = str(recipe_id)
        # Get representative caption for this recipe
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        
        # Get caption embedding
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        
        text_emb = text_embeddings[text_idx]
        similarities = image_embeddings @ text_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]  # Top 5, highest first
        
        # Get ground truth: images from this recipe_id
        gt_image_indices = set(recipe_to_image_indices.get(recipe_id_str, []))
        
        # Create figure with more vertical space for captions
        fig = plt.figure(figsize=(16, 5))
        gs = gridspec.GridSpec(1, 6, width_ratios=[2, 1, 1, 1, 1, 1], hspace=0.3, bottom=0.15)
        
        # Show query caption
        ax0 = fig.add_subplot(gs[0])
        ax0.text(0.5, 0.5, f'Query:\n"{caption}"', 
                ha='center', va='center', fontsize=14, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
        ax0.axis('off')
        
        # Show top-5 images
        for i, img_idx in enumerate(top5_indices):
            img_key = image_keys[img_idx]
            bucket, key = get_image_path_in_minio(img_key)
            img = load_image_from_minio(bucket, key)
            
            if img is None:
                continue
            
            # Check if this image is from the ground truth recipe
            is_correct = img_idx in gt_image_indices
            
            # Get recipe_id for this image to show its representative caption
            img_recipe_id = image_key_to_recipe_id.get(img_key, "Unknown")
            # Note: Shows representative caption of the recipe (longest caption), not necessarily the exact caption for this image
            img_caption = recipe_to_caption_text.get(img_recipe_id, "Unknown")
            
            ax = fig.add_subplot(gs[i + 1])
            ax.imshow(img)
            ax.axis('off')
            
            # Color border: green if correct, red if incorrect
            border_color = 'green' if is_correct else 'red'
            border_width = 3 if is_correct else 2
            for spine in ax.spines.values():
                spine.set_edgecolor(border_color)
                spine.set_linewidth(border_width)
            
            # Title with similarity score
            similarity_score = similarities[img_idx]
            rank = i + 1
            title = f"Rank {rank}\n{similarity_score:.3f}"
            if is_correct:
                title += "\n✓ Correct"
            ax.set_title(title, fontsize=10, color=border_color, fontweight='bold')
            
            # Add recipe caption below the image
            # Truncate long captions for display
            display_caption = img_caption if len(img_caption) <= 40 else img_caption[:37] + "..."
            ax.text(0.5, -0.15, f'"{display_caption}"', 
                   ha='center', va='top', fontsize=9, 
                   transform=ax.transAxes,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'),
                   wrap=True)
        
        plt.suptitle(f'Text → Image Retrieval Example {recipe_idx + 1} (Recipe: {recipe_id_str})', 
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        print()

def visualize_image_to_text_retrieval(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    recipe_to_caption_text: Dict[str, str],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    unique_captions: List[str],
    n_examples: int = 3
):
    """
    Visualize Image → Text retrieval: for each image, show top-5 captions.
    """
    
    # Sample random images
    unique_images = test_df["image_key"].unique()
    sample_images = pd.Series(unique_images).sample(
        min(n_examples, len(unique_images)),
        random_state=RANDOM_SEED
    )
    
    for img_idx_example, img_key in enumerate(sample_images):
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        similarities = text_embeddings @ img_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]  # Top 5, highest first
        
        # Get ground truth: recipe_id for this image
        gt_recipe_id_str = image_key_to_recipe_id.get(img_key)
        # Use all captions for ground truth (consistent with evaluation)
        gt_caption_indices = set(recipe_to_all_caption_indices.get(gt_recipe_id_str, [])) if gt_recipe_id_str else set()
        
        # Load query image
        bucket, key = get_image_path_in_minio(img_key)
        query_img = load_image_from_minio(bucket, key)
        
        if query_img is None:
            continue
        
        # Create figure with more vertical space for recipe name
        fig = plt.figure(figsize=(16, 4))
        gs = gridspec.GridSpec(1, 6, width_ratios=[1, 1, 1, 1, 1, 1], hspace=0.3, bottom=0.15)
        
        # Show query image
        ax0 = fig.add_subplot(gs[0])
        ax0.imshow(query_img)
        ax0.axis('off')
        ax0.set_title('Query Image', fontsize=12, fontweight='bold')
        
        # Add recipe name below the query image
        if gt_recipe_id_str:
            recipe_caption = recipe_to_caption_text.get(gt_recipe_id_str, "Unknown")
            # Truncate long captions for display
            display_recipe = recipe_caption if len(recipe_caption) <= 40 else recipe_caption[:37] + "..."
            ax0.text(0.5, -0.15, f'Recipe: {gt_recipe_id_str}\n"{display_recipe}"', 
                   ha='center', va='top', fontsize=9, 
                   transform=ax0.transAxes,
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8, edgecolor='gray'),
                   wrap=True)
        
        # Show top-5 captions
        for i, text_idx in enumerate(top5_indices):
            caption = unique_captions[text_idx]
            
            # Check if this caption is from the ground truth recipe
            is_correct = text_idx in gt_caption_indices
            
            ax = fig.add_subplot(gs[i + 1])
            ax.axis('off')
            
            # Color border: green if correct, red if incorrect
            border_color = 'green' if is_correct else 'red'
            border_width = 3 if is_correct else 2
            
            # Create text box
            similarity_score = similarities[text_idx]
            rank = i + 1
            text_content = f"Rank {rank}\n\n\"{caption}\"\n\nSimilarity: {similarity_score:.3f}"
            if is_correct:
                text_content += "\n\n✓ Correct"
            
            ax.text(0.5, 0.5, text_content,
                   ha='center', va='center', fontsize=10,
                   bbox=dict(boxstyle='round', facecolor='lightyellow', 
                            edgecolor=border_color, linewidth=border_width, alpha=0.8),
                   wrap=True)
        
        plt.suptitle(f'Image → Text Retrieval Example {img_idx_example + 1} (Recipe: {gt_recipe_id_str})', 
                     fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.show()
        print()

# Visualization functions defined above
# They will be called after embeddings are generated (see section after evaluation)


# This section has been moved to after the evaluation (section 11.5)


In [ ]:
# This cell will be moved to after the evaluation section
# The visualization functions are now in the correct location (after section 11)


In [ ]:
def clip_contrastive_loss(image_embeds, text_embeds, logit_scale):
    """
    CLIP contrastive loss with in-batch negatives.
    
    Args:
        image_embeds: Image embeddings (batch_size, embed_dim)
        text_embeds: Text embeddings (batch_size, embed_dim)
        logit_scale: Learnable temperature parameter
    
    Returns:
        loss: Contrastive loss
    """
    # Normalize embeddings
    image_embeds = F.normalize(image_embeds, dim=-1)
    text_embeds = F.normalize(text_embeds, dim=-1)
    
    # Compute logits
    logit_scale = logit_scale.exp()
    logits_per_text = logit_scale * (text_embeds @ image_embeds.T)  # (B, B)
    logits_per_image = logits_per_text.T  # (B, B)
    
    # Labels: diagonal (each text matches its corresponding image)
    batch_size = image_embeds.size(0)
    labels = torch.arange(batch_size, device=image_embeds.device)
    
    # Cross-entropy losses
    loss_t = F.cross_entropy(logits_per_text, labels)
    loss_i = F.cross_entropy(logits_per_image, labels)
    
    # Average
    loss = (loss_t + loss_i) / 2.0
    
    return loss

# Setup optimizer (only trainable parameters)
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=TRAINING_CONFIG["learning_rate"],
    weight_decay=0.01
)

# Setup learning rate scheduler (linear warmup + cosine decay)
# Calculate steps per epoch accounting for gradient accumulation
# Defensive check: ensure we have batches to train on
if len(train_dataloader) == 0:
    raise RuntimeError("train_dataloader has 0 batches. Check dataset filtering / loading.")

steps_per_epoch = int(np.ceil(len(train_dataloader) / TRAINING_CONFIG["gradient_accumulation_steps"]))
steps_per_epoch = max(1, steps_per_epoch)  # Ensure at least 1 step
total_steps = steps_per_epoch * TRAINING_CONFIG["num_epochs"]
# Ensure warmup_steps doesn't exceed total_steps
warmup_steps = min(TRAINING_CONFIG["warmup_steps"], total_steps)

def get_lr_scheduler(optimizer, num_warmup_steps, num_training_steps):
    """Create learning rate scheduler with warmup."""
    from torch.optim.lr_scheduler import LambdaLR
    
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + np.cos(np.pi * progress)))
    
    return LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, warmup_steps, total_steps)

print(f"✓ Training setup complete")
print(f"  Optimizer: AdamW (lr={TRAINING_CONFIG['learning_rate']})")
print(f"  Scheduler: Linear warmup + Cosine decay")
print(f"  Total steps: {total_steps}")
print(f"  Warmup steps: {warmup_steps}")


## 8. Training Loop


In [ ]:
# Training logs
training_logs = {
    "run_id": RUN_ID,
    "start_time": datetime.utcnow().isoformat() + "Z",
    "config": {
        "model": MODEL_NAME,
        "lora": LORA_CONFIG,
        "training": TRAINING_CONFIG,
    },
    "epochs": [],
    "total_time_seconds": 0,
    "peak_memory_mb": 0,
}

# Get device for inputs
if DEVICE == "cuda":
    main_device = next(model.parameters()).device
else:
    main_device = torch.device("cpu")

print("=" * 60)
print("Starting Training")
print("=" * 60)
print(f"Device: {main_device}")
print(f"Total epochs: {TRAINING_CONFIG['num_epochs']}")
batches_per_epoch = len(train_dataloader)
optimizer_steps_per_epoch = int(np.ceil(batches_per_epoch / TRAINING_CONFIG["gradient_accumulation_steps"]))
print(f"Batches per epoch: {batches_per_epoch}")
print(f"Optimizer steps per epoch: {optimizer_steps_per_epoch}")
print(f"Total optimizer steps: {total_steps}")
print()

model.train()
training_start_time = time.time()
global_step = 0
peak_memory = 0
step_losses = []  # Track losses per step (not per batch)

# Setup mixed precision training (AMP) for GPU
# Use generic torch.amp API (works correctly on both CPU and CUDA)
use_amp = (DEVICE == "cuda")
if use_amp:
    scaler = torch.amp.GradScaler("cuda", enabled=True)
    print("✓ Mixed precision (AMP) enabled for GPU training")
else:
    scaler = None  # No scaler needed for CPU
    print("ℹ Mixed precision disabled (CPU training)")

# Set random seeds for reproducibility
torch.manual_seed(TRAINING_CONFIG["random_seed"])
np.random.seed(TRAINING_CONFIG["random_seed"])
random.seed(TRAINING_CONFIG["random_seed"])

for epoch in range(TRAINING_CONFIG["num_epochs"]):
    epoch_start_time = time.time()
    epoch_losses = []
    running_loss = 0.0  # Accumulate loss across gradient accumulation steps
    
    print(f"\nEpoch {epoch + 1}/{TRAINING_CONFIG['num_epochs']}")
    print("-" * 60)
    
    # Reset failed image counter for this epoch
    train_dataset.failed_image_count = 0
    
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")
    
    # Track last batch_idx explicitly (more robust than using locals())
    last_batch_idx = -1
    for batch_idx, inputs in enumerate(progress_bar):
        last_batch_idx = batch_idx
        # Move inputs to device
        input_ids = inputs["input_ids"].to(main_device)
        attention_mask = inputs.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.to(main_device)
        pixel_values = inputs["pixel_values"].to(main_device)
        
        # Forward pass using specific methods (avoids PEFT wrapper issues)
        # Use get_image_features and get_text_features instead of forward()
        text_inputs = {
            "input_ids": input_ids,
        }
        if attention_mask is not None:
            text_inputs["attention_mask"] = attention_mask
        
        image_inputs = {
            "pixel_values": pixel_values,
        }
        
        # Forward pass with mixed precision (AMP) when on GPU
        # Using enabled=use_amp for robustness (works even if refactored)
        with torch.amp.autocast("cuda", enabled=use_amp):
            # Get embeddings separately
            text_embeds = model.get_text_features(**text_inputs)
            image_embeds = model.get_image_features(**image_inputs)
            
            # Compute loss
            loss = clip_contrastive_loss(
                image_embeds,
                text_embeds,
                model.logit_scale
            )
            
            # Scale loss for gradient accumulation
            loss = loss / TRAINING_CONFIG["gradient_accumulation_steps"]
        
        # Backward pass (with scaler for AMP, direct for CPU)
        if use_amp:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        
        # Accumulate unscaled loss for this accumulation step
        running_loss += float(loss.item() * TRAINING_CONFIG["gradient_accumulation_steps"])
        
        # Store unscaled loss for epoch average
        epoch_losses.append(float(loss.item() * TRAINING_CONFIG["gradient_accumulation_steps"]))
        
        # Update weights (only after accumulation steps)
        if (batch_idx + 1) % TRAINING_CONFIG["gradient_accumulation_steps"] == 0:
            if use_amp:
                # Gradient clipping (especially important with trainable logit_scale)
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(
                    [p for p in model.parameters() if p.requires_grad],
                    max_norm=1.0
                )
                scaler.step(optimizer)
                scaler.update()
            else:
                # Gradient clipping for CPU
                torch.nn.utils.clip_grad_norm_(
                    [p for p in model.parameters() if p.requires_grad],
                    max_norm=1.0
                )
                optimizer.step()
            
            # Clamp logit_scale to prevent temperature explosion (CLIP best practice)
            with torch.no_grad():
                model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)
            
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            # Store average loss for this step (across accumulation)
            step_losses.append(running_loss / TRAINING_CONFIG["gradient_accumulation_steps"])
            running_loss = 0.0
        
        # Logging (by step, not by batch)
        if global_step > 0 and global_step % TRAINING_CONFIG["logging_steps"] == 0:
            current_lr = scheduler.get_last_lr()[0]
            # Use step_losses for accurate logging
            avg_loss = np.mean(step_losses[-TRAINING_CONFIG["logging_steps"]:]) if len(step_losses) >= TRAINING_CONFIG["logging_steps"] else np.mean(step_losses)
            progress_bar.set_postfix({
                "loss": f"{avg_loss:.4f}",
                "lr": f"{current_lr:.2e}",
                "step": global_step
            })
        
        # Track peak memory
        if DEVICE == "cuda":
            current_memory = torch.cuda.max_memory_allocated() / 1024**2  # MB
            peak_memory = max(peak_memory, current_memory)
    
    # Process remaining gradients at end of epoch if not aligned with accumulation steps
    # Protection: ensure last_batch_idx is valid (should be >= 0 if dataloader has batches)
    if last_batch_idx >= 0:
        remaining_steps = (last_batch_idx + 1) % TRAINING_CONFIG["gradient_accumulation_steps"]
    else:
        remaining_steps = 0  # No batches processed
    
    if remaining_steps > 0:
        if use_amp:
            # Gradient clipping (especially important with trainable logit_scale)
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad],
                max_norm=1.0
            )
            scaler.step(optimizer)
            scaler.update()
        else:
            # Gradient clipping for CPU
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad],
                max_norm=1.0
            )
            optimizer.step()
        
        # Clamp logit_scale to prevent temperature explosion (CLIP best practice)
        with torch.no_grad():
            model.logit_scale.clamp_(LOGIT_SCALE_MIN, LOGIT_SCALE_MAX)
        
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1
        # Store average loss for remaining accumulation steps
        step_losses.append(running_loss / remaining_steps)
        running_loss = 0.0
    
    epoch_time = time.time() - epoch_start_time
    avg_epoch_loss = np.mean(epoch_losses)
    
    # Calculate optimizer steps for this epoch
    batches = len(train_dataloader)
    optimizer_steps_this_epoch = int(np.ceil(batches / TRAINING_CONFIG["gradient_accumulation_steps"]))
    
    # Report failed image loads (guardrail: detect if training is contaminated)
    failed_images_this_epoch = train_dataset.failed_image_count
    if failed_images_this_epoch > 0:
        print(f"  ⚠ Failed image loads: {failed_images_this_epoch} (training may be contaminated)")
    
    epoch_log = {
        "epoch": epoch + 1,
        "avg_loss": float(avg_epoch_loss),
        "time_seconds": float(epoch_time),
        "batches": batches,
        "optimizer_steps": optimizer_steps_this_epoch,
        "failed_image_loads": failed_images_this_epoch,  # Track for analysis
    }
    training_logs["epochs"].append(epoch_log)
    
    print(f"  Average loss: {avg_epoch_loss:.4f}")
    print(f"  Time: {epoch_time:.2f} seconds")

training_time = time.time() - training_start_time
training_logs["total_time_seconds"] = float(training_time)
training_logs["peak_memory_mb"] = float(peak_memory)
training_logs["end_time"] = datetime.utcnow().isoformat() + "Z"
training_logs["total_steps"] = global_step

print("\n" + "=" * 60)
print("Training Complete")
print("=" * 60)
print(f"Total time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"Peak memory: {peak_memory:.2f} MB")
print(f"Total steps: {global_step}")


In [ ]:
def save_json_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as JSON to MinIO."""
    try:
        json_bytes = json.dumps(data, indent=2).encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=json_bytes,
            ContentType="application/json",
        )
        size_kb = len(json_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

def save_yaml_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as YAML to MinIO."""
    try:
        yaml_str = yaml.dump(data, default_flow_style=False, sort_keys=False)
        yaml_bytes = yaml_str.encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=yaml_bytes,
            ContentType="text/yaml",
        )
        size_kb = len(yaml_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

# Save adapters locally first, then upload to MinIO
print("Saving adapters...")
local_adapters_dir = Path(f"adapters_{RUN_ID}")
model.save_pretrained(str(local_adapters_dir))

# Upload adapters to MinIO (upload config and weights - supports both .bin and .safetensors)
print("Uploading adapters to MinIO...")
adapter_files = ["adapter_config.json", "adapter_model.bin", "adapter_model.safetensors"]
for file_name in adapter_files:
    file_path = local_adapters_dir / file_name
    if file_path.exists():
        with open(file_path, "rb") as f:
            s3.put_object(
                Bucket=FINE_TUNING_BUCKET,
                Key=f"{ADAPTERS_DIR}/{file_name}",
                Body=f.read(),
            )
        print(f"  ✓ Uploaded {file_name}")

# Save training logs
print("\nSaving training logs...")
save_json_to_minio(training_logs, FINE_TUNING_BUCKET, LOGS_KEY)

# Save configuration
config_data = {
    "run_id": RUN_ID,
    "model": MODEL_NAME,
    "method": "lora",
    "created_at": datetime.utcnow().isoformat() + "Z",
    "lora_config": LORA_CONFIG,
    "training_config": TRAINING_CONFIG,
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
    },
}
save_yaml_to_minio(config_data, FINE_TUNING_BUCKET, CONFIG_KEY)

print(f"\n✅ Adapters and logs saved successfully!")
print(f"  Adapters: s3://{FINE_TUNING_BUCKET}/{ADAPTERS_DIR}/")
print(f"  Logs: s3://{FINE_TUNING_BUCKET}/{LOGS_KEY}")
print(f"  Config: s3://{FINE_TUNING_BUCKET}/{CONFIG_KEY}")


## 10. Evaluation (Same Protocol as Baseline)

Now we evaluate the fine-tuned model using the same evaluation functions as the baseline.


In [ ]:
# Set model to evaluation mode
model.eval()

# Generate embeddings for test set (same as baseline)
print("=" * 60)
print("Generating Embeddings for Evaluation")
print("=" * 60)

def generate_image_embeddings(
    df: pd.DataFrame, 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> Tuple[np.ndarray, List[str]]:
    """Generate image embeddings for all images in the dataset."""
    model.eval()
    embeddings = []
    image_keys = []
    failed_keys = []
    
    unique_images = df["image_key"].unique()
    print(f"Generating embeddings for {len(unique_images)} unique images...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(unique_images), batch_size)):
            batch_keys = unique_images[i:i+batch_size]
            batch_images = []
            valid_keys = []
            
            for img_key in batch_keys:
                bucket, key = get_image_path_in_minio(img_key)
                img = load_image_from_minio(bucket, key)
                if img is not None:
                    batch_images.append(img)
                    valid_keys.append(img_key)
                else:
                    failed_keys.append(img_key)
            
            if not batch_images:
                continue
            
            inputs = processor(images=batch_images, return_tensors="pt", padding=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_image_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            
            embeddings.append(batch_emb.cpu().numpy())
            image_keys.extend(valid_keys)
    
    if failed_keys:
        print(f"⚠ Failed to load {len(failed_keys)} images")
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(image_keys)} image embeddings (shape: {embeddings.shape})")
    return embeddings, image_keys

def generate_text_embeddings(
    texts: List[str], 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> np.ndarray:
    """Generate text embeddings for all captions."""
    model.eval()
    embeddings = []
    
    print(f"Generating embeddings for {len(texts)} texts...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]
            inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_text_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            embeddings.append(batch_emb.cpu().numpy())
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(texts)} text embeddings (shape: {embeddings.shape})")
    return embeddings

start_time = time.time()

# Generate embeddings
image_embeddings, image_keys = generate_image_embeddings(
    test_df, model, processor, main_device, batch_size=16  # Smaller batch for LoRA
)

unique_captions = test_df["caption"].unique().tolist()
text_embeddings = generate_text_embeddings(unique_captions, model, processor, main_device, batch_size=16)

# Create official copy to ensure alignment with text_embeddings (never modify this)
unique_captions_list = list(unique_captions)  # Explicit copy to prevent accidental modification

embedding_time = time.time() - start_time
print(f"\n✓ Embedding generation complete in {embedding_time:.2f} seconds")

# Create mapping dictionaries
image_key_to_idx = {key: idx for idx, key in enumerate(image_keys)}
caption_to_idx = {caption: idx for idx, caption in enumerate(unique_captions_list)}

# Precompute lookup dictionaries for O(1) access (same as baseline)
print("\nPrecomputing lookup dictionaries...")

# recipe_id -> list of image indices
recipe_to_image_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_images = test_df[test_df["recipe_id"] == recipe_id]["image_key"].unique()
    recipe_to_image_indices[recipe_id_str] = [
        image_key_to_idx.get(img_key) 
        for img_key in recipe_images 
        if image_key_to_idx.get(img_key) is not None
    ]

# recipe_id -> list of caption indices (one caption per recipe for text→image query)
# Use longest caption as representative (most informative)
recipe_to_caption_indices: Dict[str, List[int]] = {}
recipe_to_caption_text: Dict[str, str] = {}
# recipe_id -> list of ALL caption indices (for image→text ground truth)
recipe_to_all_caption_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_captions = test_df[test_df["recipe_id"] == recipe_id]["caption"].unique()
    if len(recipe_captions) == 0:
        continue
    
    # Use longest caption as representative (most informative)
    # If multiple have same length, use first after sorting alphabetically (deterministic)
    representative_caption = max(recipe_captions, key=lambda c: (len(c), c))
    
    # Store representative caption for text→image queries
    cap_idx = caption_to_idx.get(representative_caption)
    if cap_idx is not None:
        recipe_to_caption_indices[recipe_id_str] = [cap_idx]
        recipe_to_caption_text[recipe_id_str] = representative_caption
    
    # Store ALL captions for image→text ground truth (more fair if multiple captions per recipe)
    all_caption_indices = []
    for caption in recipe_captions:
        cap_idx = caption_to_idx.get(caption)
        if cap_idx is not None:
            all_caption_indices.append(cap_idx)
    recipe_to_all_caption_indices[recipe_id_str] = sorted(set(all_caption_indices))  # Remove duplicates, sorted for determinism

# image_key -> recipe_id (as string for consistency)
image_key_to_recipe_id: Dict[str, str] = {}
for _, row in test_df.iterrows():
    if row["image_key"] not in image_key_to_recipe_id:
        image_key_to_recipe_id[row["image_key"]] = str(row["recipe_id"])

print(f"✓ Precomputed lookups for {len(recipe_to_image_indices)} recipes")


## 11. Evaluation Functions (Same as Baseline)


In [ ]:
# Copy evaluation functions from baseline (same protocol by recipe_id)
def compute_recall_at_k(scores: np.ndarray, ground_truth_indices: List[int], k: int) -> float:
    """
    Compute Recall@K.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
        k: top-K to consider
    
    Returns:
        recall@k: 1.0 if any ground truth is in top-K, else 0.0
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    top_k_indices = np.argsort(scores)[-k:][::-1]  # Top K, highest first
    top_k_set = set(top_k_indices)
    
    # Check if any ground truth is in top-K
    for gt_idx in ground_truth_indices:
        if gt_idx in top_k_set:
            return 1.0
    
    return 0.0

def compute_mrr(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute Mean Reciprocal Rank.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        mrr: 1/rank of first correct result, or 0.0 if none found
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return 1.0 / rank
    
    return 0.0

def compute_first_hit_rank(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute rank of first correct result (first hit rank).
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        first_hit_rank: rank of first correct result (1-indexed), or len(scores)+1 if not found
    """
    if len(ground_truth_indices) == 0:
        return float(len(scores) + 1)
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return float(rank)
    
    return float(len(scores) + 1)  # Not found

def evaluate_text_to_image_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_indices: Dict[str, List[int]],
    recipe_to_image_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Text → Image retrieval by recipe_id.
    
    For each recipe_id, use its representative caption to retrieve images.
    Ground truth: all images from that recipe_id.
    """
    print("\nEvaluating Text → Image retrieval (by recipe_id)...")
    
    unique_recipe_ids = test_df["recipe_id"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for recipe_id in tqdm(unique_recipe_ids, desc="Processing recipes"):
        recipe_id_str = str(recipe_id)
        # Get representative caption embedding (first caption for this recipe)
        caption_indices = recipe_to_caption_indices.get(recipe_id_str, [])
        if not caption_indices:
            continue
        
        # Use first caption (or could average multiple)
        text_idx = caption_indices[0]
        text_emb = text_embeddings[text_idx]
        
        # Compute similarities with all images
        similarities = image_embeddings @ text_emb  # (n_images,)
        
        # Get ground truth: all images from this recipe_id
        ground_truth_indices = recipe_to_image_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

def evaluate_image_to_text_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Image → Text retrieval by recipe_id.
    
    For each image, retrieve captions. Ground truth: ALL captions from the same recipe_id.
    """
    print("\nEvaluating Image → Text retrieval (by recipe_id)...")
    
    unique_images = test_df["image_key"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for img_key in tqdm(unique_images, desc="Processing images"):
        # Get image embedding
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        
        # Compute similarities with all captions
        similarities = text_embeddings @ img_emb  # (n_texts,)
        
        # Get recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: ALL captions from the same recipe_id (more fair)
        ground_truth_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

# Run evaluation
print("=" * 60)
print("Running Evaluation")
print("=" * 60)

eval_start_time = time.time()

# Text → Image retrieval (by recipe_id)
text_to_image_results = evaluate_text_to_image_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_indices,
    recipe_to_image_indices,
    image_key_to_idx,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

# Image → Text retrieval (by recipe_id)
image_to_text_results = evaluate_image_to_text_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    image_key_to_idx,
    image_key_to_recipe_id,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

eval_time = time.time() - eval_start_time

print("\n" + "=" * 60)
print("Evaluation Results")
print("=" * 60)

# Mapping for display names (only for specific metrics)
METRIC_DISPLAY_NAMES = {
    "MedianRank_first_hit": "MedianRank",
    "MeanRank_first_hit": "MeanRank"
}

print("\n📊 Text → Image Retrieval:")
for metric, value in text_to_image_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print("\n📊 Image → Text Retrieval:")
for metric, value in image_to_text_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print(f"\n⏱ Evaluation time: {eval_time:.2f} seconds")


## 11.5. Visualize Retrieval Results

Visualize the retrieval results to see how well the fine-tuned model performs:
- **Text → Image**: For each recipe, show the top-5 retrieved images
- **Image → Text**: For each image, show the top-5 retrieved captions


In [ ]:
# Visualize results
print("=" * 60)
print("Visualizing Retrieval Results")
print("=" * 60)

# Verify required variables exist (safety check)
assert "image_key_to_recipe_id" in globals(), "image_key_to_recipe_id not found. Run evaluation section first."
assert "text_embeddings" in globals(), "text_embeddings not found. Run evaluation section first."
assert "image_embeddings" in globals(), "image_embeddings not found. Run evaluation section first."
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."
assert "image_keys" in globals(), "image_keys not found. Run evaluation section first."

print("\n📝 Text → Image Retrieval (Top-5 images for each recipe):")
print("-" * 60)
visualize_text_to_image_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=3
)

print("\n🖼️ Image → Text Retrieval (Top-5 captions for each image):")
print("-" * 60)
visualize_image_to_text_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions_list,
    n_examples=3
)


## 12. Generate Qualitative Examples


In [ ]:
def generate_top5_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_text: Dict[str, str],
    recipe_to_image_indices: Dict[str, List[int]],
    caption_to_idx: Dict[str, int],
    image_keys: List[str],
    image_key_to_recipe_id: Dict[str, str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 retrievals by recipe_id.
    """
    
    examples = []
    
    # Filter recipes with valid ground truth before sampling (avoid bias)
    valid_recipe_ids = []
    for recipe_id in test_df["recipe_id"].unique():
        recipe_id_str = str(recipe_id)
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        gt_image_indices = recipe_to_image_indices.get(recipe_id_str, [])
        if len(gt_image_indices) > 0:  # Only include if has valid GT
            valid_recipe_ids.append(recipe_id)
    
    # Sample random recipes from valid ones
    if len(valid_recipe_ids) == 0:
        print("⚠ No valid recipes with ground truth images found")
        return examples
    
    sample_recipe_ids = pd.Series(valid_recipe_ids).sample(
        min(n_examples, len(valid_recipe_ids)),
        random_state=RANDOM_SEED
    )
    
    for recipe_id in sample_recipe_ids:
        recipe_id_str = str(recipe_id)
        # Get representative caption for this recipe
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        
        # Get caption embedding
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        
        text_emb = text_embeddings[text_idx]
        similarities = image_embeddings @ text_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: images from this recipe_id
        gt_image_indices = set(recipe_to_image_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved images
        top5_images = []
        for i, idx in enumerate(top5_indices):
            img_key = image_keys[idx]
            img_recipe_id = image_key_to_recipe_id.get(img_key)
            is_correct = idx in gt_image_indices
            
            top5_images.append({
                "rank": i + 1,
                "image_key": img_key,
                "recipe_id": img_recipe_id,
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "text_to_image",
            "recipe_id": recipe_id_str,
            "query_caption": caption,
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_images
        })
    
    return examples

def generate_image_to_text_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    recipe_to_caption_text: Dict[str, str],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    unique_captions: List[str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 captions for images (Image→Text retrieval).
    """
    examples = []
    
    # Filter images with valid ground truth before sampling (avoid bias)
    valid_images = []
    for img_key in test_df["image_key"].unique():
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        gt_caption_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        if len(gt_caption_indices) > 0:  # Only include if has valid GT
            valid_images.append(img_key)
    
    # Sample random images from valid ones
    if len(valid_images) == 0:
        print("⚠ No valid images with ground truth captions found")
        return examples
    
    sample_images = pd.Series(valid_images).sample(
        min(n_examples, len(valid_images)),
        random_state=RANDOM_SEED
    )
    
    for img_key in sample_images:
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        similarities = text_embeddings @ img_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: all captions from this recipe_id
        gt_caption_indices = set(recipe_to_all_caption_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved captions
        top5_captions = []
        for i, idx in enumerate(top5_indices):
            caption = unique_captions[idx]
            is_correct = idx in gt_caption_indices
            
            top5_captions.append({
                "rank": i + 1,
                "caption": caption,
                "caption_index": int(idx),
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "image_to_text",
            "recipe_id": recipe_id_str,
            "query_image_key": img_key,
            "recipe_representative_caption": recipe_to_caption_text.get(recipe_id_str),
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_captions
        })
    
    return examples

# Generate examples
N_QUALITATIVE_EXAMPLES = 5
print("Generating qualitative examples...")

# Verify unique_captions_list exists (should be from evaluation section)
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."

# Text → Image examples
text_to_image_examples = generate_top5_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=N_QUALITATIVE_EXAMPLES
)

# Image → Text examples
image_to_text_examples = generate_image_to_text_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions=unique_captions_list,
    n_examples=N_QUALITATIVE_EXAMPLES
)

print(f"✓ Generated {len(text_to_image_examples)} text→image examples (requested: {N_QUALITATIVE_EXAMPLES})")
print(f"✓ Generated {len(image_to_text_examples)} image→text examples (requested: {N_QUALITATIVE_EXAMPLES})")


## 13. Save Results


In [ ]:
# Create results dictionary
results = {
    "metadata": {
        "run_id": RUN_ID,
        "model": MODEL_NAME,
        "method": "lora",
        "created_at": datetime.utcnow().isoformat() + "Z",
        "device": str(main_device),
        "train_dataset": TRAIN_AUGMENTED_KEY,
        "test_dataset": TEST_MANIFEST_KEY,
        "filter_applied": "label == 1 (positive pairs only) for training",
    },
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
        "frozen": False,
    },
    "training_info": {
        "total_time_seconds": training_logs["total_time_seconds"],
        "peak_memory_mb": training_logs["peak_memory_mb"],
        "total_steps": training_logs["total_steps"],
        "num_epochs": TRAINING_CONFIG["num_epochs"],
        "final_loss": float(training_logs["epochs"][-1]["avg_loss"]) if training_logs["epochs"] else None,
    },
    "dataset_info": {
        "test_pairs": len(test_df),
        "test_recipes": test_df["recipe_id"].nunique(),
        "test_images": test_df["image_key"].nunique(),
        "test_captions": test_df["caption"].nunique(),
    },
    "metrics": {
        "text_to_image": text_to_image_results,
        "image_to_text": image_to_text_results,
    },
    "timing": {
        "embedding_generation_seconds": embedding_time,
        "evaluation_seconds": eval_time,
        "total_seconds": embedding_time + eval_time,
    },
}

# Save results
print("=" * 60)
print("Saving Results to MinIO")
print("=" * 60)

save_json_to_minio(results, FINE_TUNING_BUCKET, RESULTS_KEY)
save_json_to_minio({
    "text_to_image_examples": text_to_image_examples,
    "image_to_text_examples": image_to_text_examples
}, FINE_TUNING_BUCKET, EXAMPLES_KEY)

print(f"\n✅ Results saved successfully!")
print(f"  Results: s3://{FINE_TUNING_BUCKET}/{RESULTS_KEY}")
print(f"  Examples: s3://{FINE_TUNING_BUCKET}/{EXAMPLES_KEY}")

# Print summary
print("\n" + "=" * 60)
print("Summary")
print("=" * 60)
print(f"Run ID: {RUN_ID}")
print(f"Model: {MODEL_NAME} (LoRA fine-tuned)")
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Peak memory: {peak_memory:.2f} MB")
print(f"\nText → Image:")
print(f"  R@1: {text_to_image_results.get('R@1', 0):.4f}")
print(f"  R@5: {text_to_image_results.get('R@5', 0):.4f}")
print(f"  R@10: {text_to_image_results.get('R@10', 0):.4f}")
if "MRR" in text_to_image_results:
    print(f"  MRR: {text_to_image_results['MRR']:.4f}")
print(f"\nImage → Text:")
print(f"  R@1: {image_to_text_results.get('R@1', 0):.4f}")
print(f"  R@5: {image_to_text_results.get('R@5', 0):.4f}")
print(f"  R@10: {image_to_text_results.get('R@10', 0):.4f}")
if "MRR" in image_to_text_results:
    print(f"  MRR: {image_to_text_results['MRR']:.4f}")
print(f"\n📁 All outputs saved to: s3://{FINE_TUNING_BUCKET}/{RUN_DIR}/")


In [ ]:
def save_json_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as JSON to MinIO."""
    try:
        json_bytes = json.dumps(data, indent=2).encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=json_bytes,
            ContentType="application/json",
        )
        size_kb = len(json_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

def save_yaml_to_minio(data: Dict, bucket: str, key: str) -> bool:
    """Save dictionary as YAML to MinIO."""
    try:
        yaml_str = yaml.dump(data, default_flow_style=False, sort_keys=False)
        yaml_bytes = yaml_str.encode("utf-8")
        s3.put_object(
            Bucket=bucket,
            Key=key,
            Body=yaml_bytes,
            ContentType="text/yaml",
        )
        size_kb = len(yaml_bytes) / 1024
        print(f"✓ Saved to s3://{bucket}/{key} ({size_kb:.1f} KB)")
        return True
    except Exception as e:
        print(f"✗ Failed to save: {e}")
        return False

# Save adapters locally first, then upload to MinIO
print("Saving adapters...")
local_adapters_dir = Path(f"adapters_{RUN_ID}")
model.save_pretrained(str(local_adapters_dir))

# Upload adapters to MinIO (upload config and weights - supports both .bin and .safetensors)
print("Uploading adapters to MinIO...")
adapter_files = ["adapter_config.json", "adapter_model.bin", "adapter_model.safetensors"]
for file_name in adapter_files:
    file_path = local_adapters_dir / file_name
    if file_path.exists():
        with open(file_path, "rb") as f:
            s3.put_object(
                Bucket=FINE_TUNING_BUCKET,
                Key=f"{ADAPTERS_DIR}/{file_name}",
                Body=f.read(),
            )
        print(f"  ✓ Uploaded {file_name}")

# Save training logs
print("\nSaving training logs...")
save_json_to_minio(training_logs, FINE_TUNING_BUCKET, LOGS_KEY)

# Save configuration
config_data = {
    "run_id": RUN_ID,
    "model": MODEL_NAME,
    "method": "lora",
    "created_at": datetime.utcnow().isoformat() + "Z",
    "lora_config": LORA_CONFIG,
    "training_config": TRAINING_CONFIG,
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
    },
}
save_yaml_to_minio(config_data, FINE_TUNING_BUCKET, CONFIG_KEY)

print(f"\n✅ Adapters and logs saved successfully!")
print(f"  Adapters: s3://{FINE_TUNING_BUCKET}/{ADAPTERS_DIR}/")
print(f"  Logs: s3://{FINE_TUNING_BUCKET}/{LOGS_KEY}")
print(f"  Config: s3://{FINE_TUNING_BUCKET}/{CONFIG_KEY}")


## 10. Evaluation (Same Protocol as Baseline)

Now we evaluate the fine-tuned model using the same evaluation functions as the baseline.


In [ ]:
# Set model to evaluation mode
model.eval()

# Generate embeddings for test set (same as baseline)
print("=" * 60)
print("Generating Embeddings for Evaluation")
print("=" * 60)

def generate_image_embeddings(
    df: pd.DataFrame, 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> Tuple[np.ndarray, List[str]]:
    """Generate image embeddings for all images in the dataset."""
    model.eval()
    embeddings = []
    image_keys = []
    failed_keys = []
    
    unique_images = df["image_key"].unique()
    print(f"Generating embeddings for {len(unique_images)} unique images...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(unique_images), batch_size)):
            batch_keys = unique_images[i:i+batch_size]
            batch_images = []
            valid_keys = []
            
            for img_key in batch_keys:
                bucket, key = get_image_path_in_minio(img_key)
                img = load_image_from_minio(bucket, key)
                if img is not None:
                    batch_images.append(img)
                    valid_keys.append(img_key)
                else:
                    failed_keys.append(img_key)
            
            if not batch_images:
                continue
            
            inputs = processor(images=batch_images, return_tensors="pt", padding=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_image_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            
            embeddings.append(batch_emb.cpu().numpy())
            image_keys.extend(valid_keys)
    
    if failed_keys:
        print(f"⚠ Failed to load {len(failed_keys)} images")
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(image_keys)} image embeddings (shape: {embeddings.shape})")
    return embeddings, image_keys

def generate_text_embeddings(
    texts: List[str], 
    model, 
    processor, 
    device: Union[str, torch.device], 
    batch_size: int = 32
) -> np.ndarray:
    """Generate text embeddings for all captions."""
    model.eval()
    embeddings = []
    
    print(f"Generating embeddings for {len(texts)} texts...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]
            inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True)
            # Move inputs to device explicitly (more robust than .to(device) on BatchEncoding)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model.get_text_features(**inputs)
            batch_emb = F.normalize(outputs, p=2, dim=1)
            embeddings.append(batch_emb.cpu().numpy())
    
    embeddings = np.vstack(embeddings) if embeddings else np.array([])
    print(f"✓ Generated {len(texts)} text embeddings (shape: {embeddings.shape})")
    return embeddings

start_time = time.time()

# Generate embeddings
image_embeddings, image_keys = generate_image_embeddings(
    test_df, model, processor, main_device, batch_size=16  # Smaller batch for LoRA
)

unique_captions = test_df["caption"].unique().tolist()
text_embeddings = generate_text_embeddings(unique_captions, model, processor, main_device, batch_size=16)

# Create official copy to ensure alignment with text_embeddings (never modify this)
unique_captions_list = list(unique_captions)  # Explicit copy to prevent accidental modification

embedding_time = time.time() - start_time
print(f"\n✓ Embedding generation complete in {embedding_time:.2f} seconds")

# Create mapping dictionaries
image_key_to_idx = {key: idx for idx, key in enumerate(image_keys)}
caption_to_idx = {caption: idx for idx, caption in enumerate(unique_captions_list)}

# Precompute lookup dictionaries for O(1) access (same as baseline)
print("\nPrecomputing lookup dictionaries...")

# recipe_id -> list of image indices
recipe_to_image_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_images = test_df[test_df["recipe_id"] == recipe_id]["image_key"].unique()
    recipe_to_image_indices[recipe_id_str] = [
        image_key_to_idx.get(img_key) 
        for img_key in recipe_images 
        if image_key_to_idx.get(img_key) is not None
    ]

# recipe_id -> list of caption indices (one caption per recipe for text→image query)
# Use longest caption as representative (most informative)
recipe_to_caption_indices: Dict[str, List[int]] = {}
recipe_to_caption_text: Dict[str, str] = {}
# recipe_id -> list of ALL caption indices (for image→text ground truth)
recipe_to_all_caption_indices: Dict[str, List[int]] = {}
for recipe_id in test_df["recipe_id"].unique():
    recipe_id_str = str(recipe_id)
    recipe_captions = test_df[test_df["recipe_id"] == recipe_id]["caption"].unique()
    if len(recipe_captions) == 0:
        continue
    
    # Use longest caption as representative (most informative)
    # If multiple have same length, use first after sorting alphabetically (deterministic)
    representative_caption = max(recipe_captions, key=lambda c: (len(c), c))
    
    # Store representative caption for text→image queries
    cap_idx = caption_to_idx.get(representative_caption)
    if cap_idx is not None:
        recipe_to_caption_indices[recipe_id_str] = [cap_idx]
        recipe_to_caption_text[recipe_id_str] = representative_caption
    
    # Store ALL captions for image→text ground truth (more fair if multiple captions per recipe)
    all_caption_indices = []
    for caption in recipe_captions:
        cap_idx = caption_to_idx.get(caption)
        if cap_idx is not None:
            all_caption_indices.append(cap_idx)
    recipe_to_all_caption_indices[recipe_id_str] = sorted(set(all_caption_indices))  # Remove duplicates, sorted for determinism

# image_key -> recipe_id (as string for consistency)
image_key_to_recipe_id: Dict[str, str] = {}
for _, row in test_df.iterrows():
    if row["image_key"] not in image_key_to_recipe_id:
        image_key_to_recipe_id[row["image_key"]] = str(row["recipe_id"])

print(f"✓ Precomputed lookups for {len(recipe_to_image_indices)} recipes")


## 11. Evaluation Functions (Same as Baseline)


In [ ]:
# Copy evaluation functions from baseline (same protocol by recipe_id)
def compute_recall_at_k(scores: np.ndarray, ground_truth_indices: List[int], k: int) -> float:
    """
    Compute Recall@K.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
        k: top-K to consider
    
    Returns:
        recall@k: 1.0 if any ground truth is in top-K, else 0.0
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    top_k_indices = np.argsort(scores)[-k:][::-1]  # Top K, highest first
    top_k_set = set(top_k_indices)
    
    # Check if any ground truth is in top-K
    for gt_idx in ground_truth_indices:
        if gt_idx in top_k_set:
            return 1.0
    
    return 0.0

def compute_mrr(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute Mean Reciprocal Rank.
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        mrr: 1/rank of first correct result, or 0.0 if none found
    """
    if len(ground_truth_indices) == 0:
        return 0.0
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return 1.0 / rank
    
    return 0.0

def compute_first_hit_rank(scores: np.ndarray, ground_truth_indices: List[int]) -> float:
    """
    Compute rank of first correct result (first hit rank).
    
    Args:
        scores: similarity scores for all candidates (1D array)
        ground_truth_indices: list of correct candidate indices
    
    Returns:
        first_hit_rank: rank of first correct result (1-indexed), or len(scores)+1 if not found
    """
    if len(ground_truth_indices) == 0:
        return float(len(scores) + 1)
    
    # Sort by score (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    # Find rank of first ground truth
    for rank, idx in enumerate(sorted_indices, start=1):
        if idx in ground_truth_indices:
            return float(rank)
    
    return float(len(scores) + 1)  # Not found

def evaluate_text_to_image_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_indices: Dict[str, List[int]],
    recipe_to_image_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Text → Image retrieval by recipe_id.
    
    For each recipe_id, use its representative caption to retrieve images.
    Ground truth: all images from that recipe_id.
    """
    print("\nEvaluating Text → Image retrieval (by recipe_id)...")
    
    unique_recipe_ids = test_df["recipe_id"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for recipe_id in tqdm(unique_recipe_ids, desc="Processing recipes"):
        recipe_id_str = str(recipe_id)
        # Get representative caption embedding (first caption for this recipe)
        caption_indices = recipe_to_caption_indices.get(recipe_id_str, [])
        if not caption_indices:
            continue
        
        # Use first caption (or could average multiple)
        text_idx = caption_indices[0]
        text_emb = text_embeddings[text_idx]
        
        # Compute similarities with all images
        similarities = image_embeddings @ text_emb  # (n_images,)
        
        # Get ground truth: all images from this recipe_id
        ground_truth_indices = recipe_to_image_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

def evaluate_image_to_text_by_recipe(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    k_values: List[int],
    should_compute_mrr: bool = True,
    should_compute_median_rank: bool = True
) -> Dict:
    """
    Evaluate Image → Text retrieval by recipe_id.
    
    For each image, retrieve captions. Ground truth: ALL captions from the same recipe_id.
    """
    print("\nEvaluating Image → Text retrieval (by recipe_id)...")
    
    unique_images = test_df["image_key"].unique()
    
    recalls = {f"R@{k}": [] for k in k_values}
    mrr_scores = [] if should_compute_mrr else None
    median_ranks = [] if should_compute_median_rank else None
    
    for img_key in tqdm(unique_images, desc="Processing images"):
        # Get image embedding
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        
        # Compute similarities with all captions
        similarities = text_embeddings @ img_emb  # (n_texts,)
        
        # Get recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: ALL captions from the same recipe_id (more fair)
        ground_truth_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        ground_truth_indices = [idx for idx in ground_truth_indices if idx is not None]
        
        if len(ground_truth_indices) == 0:
            continue
        
        # Compute metrics
        for k in k_values:
            recall = compute_recall_at_k(similarities, ground_truth_indices, k)
            recalls[f"R@{k}"].append(recall)
        
        if should_compute_mrr:
            mrr = compute_mrr(similarities, ground_truth_indices)
            mrr_scores.append(mrr)
        
        if should_compute_median_rank:
            first_hit_rank = compute_first_hit_rank(similarities, ground_truth_indices)
            median_ranks.append(first_hit_rank)
    
    # Average metrics
    results = {}
    for k in k_values:
        results[f"R@{k}"] = float(np.mean(recalls[f"R@{k}"])) if recalls[f"R@{k}"] else 0.0
    
    if should_compute_mrr and mrr_scores:
        results["MRR"] = float(np.mean(mrr_scores))
    
    if should_compute_median_rank and median_ranks:
        results["MedianRank_first_hit"] = float(np.median(median_ranks))
        results["MeanRank_first_hit"] = float(np.mean(median_ranks))
    
    return results

# Run evaluation
print("=" * 60)
print("Running Evaluation")
print("=" * 60)

eval_start_time = time.time()

# Text → Image retrieval (by recipe_id)
text_to_image_results = evaluate_text_to_image_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_indices,
    recipe_to_image_indices,
    image_key_to_idx,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

# Image → Text retrieval (by recipe_id)
image_to_text_results = evaluate_image_to_text_by_recipe(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    image_key_to_idx,
    image_key_to_recipe_id,
    K_VALUES,
    COMPUTE_MRR,
    should_compute_median_rank=True
)

eval_time = time.time() - eval_start_time

print("\n" + "=" * 60)
print("Evaluation Results")
print("=" * 60)

# Mapping for display names (only for specific metrics)
METRIC_DISPLAY_NAMES = {
    "MedianRank_first_hit": "MedianRank",
    "MeanRank_first_hit": "MeanRank"
}

print("\n📊 Text → Image Retrieval:")
for metric, value in text_to_image_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print("\n📊 Image → Text Retrieval:")
for metric, value in image_to_text_results.items():
    # Display with shorter names for readability (only for specific metrics)
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    # Format ranks with 2 decimals, other metrics with 4 decimals
    if metric in ("MedianRank_first_hit", "MeanRank_first_hit"):
        print(f"  {display_name}: {value:.2f}")
    else:
        print(f"  {display_name}: {value:.4f}")

print(f"\n⏱ Evaluation time: {eval_time:.2f} seconds")


## 11.5. Visualize Retrieval Results

Visualize the retrieval results to see how well the fine-tuned model performs:
- **Text → Image**: For each recipe, show the top-5 retrieved images
- **Image → Text**: For each image, show the top-5 retrieved captions


In [ ]:
# Visualize results
print("=" * 60)
print("Visualizing Retrieval Results")
print("=" * 60)

# Verify required variables exist (safety check)
assert "image_key_to_recipe_id" in globals(), "image_key_to_recipe_id not found. Run evaluation section first."
assert "text_embeddings" in globals(), "text_embeddings not found. Run evaluation section first."
assert "image_embeddings" in globals(), "image_embeddings not found. Run evaluation section first."
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."
assert "image_keys" in globals(), "image_keys not found. Run evaluation section first."

print("\n📝 Text → Image Retrieval (Top-5 images for each recipe):")
print("-" * 60)
visualize_text_to_image_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=3
)

print("\n🖼️ Image → Text Retrieval (Top-5 captions for each image):")
print("-" * 60)
visualize_image_to_text_retrieval(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions_list,
    n_examples=3
)


## 12. Generate Qualitative Examples


In [ ]:
def generate_top5_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_caption_text: Dict[str, str],
    recipe_to_image_indices: Dict[str, List[int]],
    caption_to_idx: Dict[str, int],
    image_keys: List[str],
    image_key_to_recipe_id: Dict[str, str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 retrievals by recipe_id.
    """
    
    examples = []
    
    # Filter recipes with valid ground truth before sampling (avoid bias)
    valid_recipe_ids = []
    for recipe_id in test_df["recipe_id"].unique():
        recipe_id_str = str(recipe_id)
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        gt_image_indices = recipe_to_image_indices.get(recipe_id_str, [])
        if len(gt_image_indices) > 0:  # Only include if has valid GT
            valid_recipe_ids.append(recipe_id)
    
    # Sample random recipes from valid ones
    if len(valid_recipe_ids) == 0:
        print("⚠ No valid recipes with ground truth images found")
        return examples
    
    sample_recipe_ids = pd.Series(valid_recipe_ids).sample(
        min(n_examples, len(valid_recipe_ids)),
        random_state=RANDOM_SEED
    )
    
    for recipe_id in sample_recipe_ids:
        recipe_id_str = str(recipe_id)
        # Get representative caption for this recipe
        caption = recipe_to_caption_text.get(recipe_id_str)
        if not caption:
            continue
        
        # Get caption embedding
        text_idx = caption_to_idx.get(caption)
        if text_idx is None:
            continue
        
        text_emb = text_embeddings[text_idx]
        similarities = image_embeddings @ text_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: images from this recipe_id
        gt_image_indices = set(recipe_to_image_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved images
        top5_images = []
        for i, idx in enumerate(top5_indices):
            img_key = image_keys[idx]
            img_recipe_id = image_key_to_recipe_id.get(img_key)
            is_correct = idx in gt_image_indices
            
            top5_images.append({
                "rank": i + 1,
                "image_key": img_key,
                "recipe_id": img_recipe_id,
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "text_to_image",
            "recipe_id": recipe_id_str,
            "query_caption": caption,
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_images
        })
    
    return examples

def generate_image_to_text_examples(
    test_df: pd.DataFrame,
    text_embeddings: np.ndarray,
    image_embeddings: np.ndarray,
    recipe_to_all_caption_indices: Dict[str, List[int]],
    recipe_to_caption_text: Dict[str, str],
    image_key_to_idx: Dict[str, int],
    image_key_to_recipe_id: Dict[str, str],
    unique_captions: List[str],
    n_examples: int = 5
) -> List[Dict]:
    """
    Generate qualitative examples showing top-5 captions for images (Image→Text retrieval).
    """
    examples = []
    
    # Filter images with valid ground truth before sampling (avoid bias)
    valid_images = []
    for img_key in test_df["image_key"].unique():
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        gt_caption_indices = recipe_to_all_caption_indices.get(recipe_id_str, [])
        if len(gt_caption_indices) > 0:  # Only include if has valid GT
            valid_images.append(img_key)
    
    # Sample random images from valid ones
    if len(valid_images) == 0:
        print("⚠ No valid images with ground truth captions found")
        return examples
    
    sample_images = pd.Series(valid_images).sample(
        min(n_examples, len(valid_images)),
        random_state=RANDOM_SEED
    )
    
    for img_key in sample_images:
        img_idx = image_key_to_idx.get(img_key)
        if img_idx is None:
            continue
        
        img_emb = image_embeddings[img_idx]
        similarities = text_embeddings @ img_emb
        top5_indices = np.argsort(similarities)[-5:][::-1]
        
        # Get ground truth: recipe_id for this image
        recipe_id_str = image_key_to_recipe_id.get(img_key)
        if not recipe_id_str:
            continue
        
        # Get ground truth: all captions from this recipe_id
        gt_caption_indices = set(recipe_to_all_caption_indices.get(recipe_id_str, []))
        
        # Get top-5 retrieved captions
        top5_captions = []
        for i, idx in enumerate(top5_indices):
            caption = unique_captions[idx]
            is_correct = idx in gt_caption_indices
            
            top5_captions.append({
                "rank": i + 1,
                "caption": caption,
                "caption_index": int(idx),
                "similarity": float(similarities[idx]),
                "is_correct": is_correct
            })
        
        examples.append({
            "query_type": "image_to_text",
            "recipe_id": recipe_id_str,
            "query_image_key": img_key,
            "recipe_representative_caption": recipe_to_caption_text.get(recipe_id_str),
            "ground_truth_recipe_id": recipe_id_str,
            "top5_results": top5_captions
        })
    
    return examples

# Generate examples
N_QUALITATIVE_EXAMPLES = 5
print("Generating qualitative examples...")

# Verify unique_captions_list exists (should be from evaluation section)
assert "unique_captions_list" in globals(), "unique_captions_list not found. Run evaluation section first."

# Text → Image examples
text_to_image_examples = generate_top5_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_caption_text,
    recipe_to_image_indices,
    caption_to_idx,
    image_keys,
    image_key_to_recipe_id,
    n_examples=N_QUALITATIVE_EXAMPLES
)

# Image → Text examples
image_to_text_examples = generate_image_to_text_examples(
    test_df,
    text_embeddings,
    image_embeddings,
    recipe_to_all_caption_indices,
    recipe_to_caption_text,
    image_key_to_idx,
    image_key_to_recipe_id,
    unique_captions=unique_captions_list,
    n_examples=N_QUALITATIVE_EXAMPLES
)

print(f"✓ Generated {len(text_to_image_examples)} text→image examples (requested: {N_QUALITATIVE_EXAMPLES})")
print(f"✓ Generated {len(image_to_text_examples)} image→text examples (requested: {N_QUALITATIVE_EXAMPLES})")


## 13. Save Results


In [ ]:
# Create results dictionary
results = {
    "metadata": {
        "run_id": RUN_ID,
        "model": MODEL_NAME,
        "method": "lora",
        "created_at": datetime.utcnow().isoformat() + "Z",
        "device": str(main_device),
        "train_dataset": TRAIN_AUGMENTED_KEY,
        "test_dataset": TEST_MANIFEST_KEY,
        "filter_applied": "label == 1 (positive pairs only) for training",
    },
    "model_info": {
        "total_parameters": int(total_params),
        "trainable_parameters": int(trainable_params),
        "trainable_ratio": float(trainable_params / total_params),
        "frozen": False,
    },
    "training_info": {
        "total_time_seconds": training_logs["total_time_seconds"],
        "peak_memory_mb": training_logs["peak_memory_mb"],
        "total_steps": training_logs["total_steps"],
        "num_epochs": TRAINING_CONFIG["num_epochs"],
        "final_loss": float(training_logs["epochs"][-1]["avg_loss"]) if training_logs["epochs"] else None,
    },
    "dataset_info": {
        "test_pairs": len(test_df),
        "test_recipes": test_df["recipe_id"].nunique(),
        "test_images": test_df["image_key"].nunique(),
        "test_captions": test_df["caption"].nunique(),
    },
    "metrics": {
        "text_to_image": text_to_image_results,
        "image_to_text": image_to_text_results,
    },
    "timing": {
        "embedding_generation_seconds": embedding_time,
        "evaluation_seconds": eval_time,
        "total_seconds": embedding_time + eval_time,
    },
}

# Save results
print("=" * 60)
print("Saving Results to MinIO")
print("=" * 60)

save_json_to_minio(results, FINE_TUNING_BUCKET, RESULTS_KEY)
save_json_to_minio({
    "text_to_image_examples": text_to_image_examples,
    "image_to_text_examples": image_to_text_examples
}, FINE_TUNING_BUCKET, EXAMPLES_KEY)

print(f"\n✅ Results saved successfully!")
print(f"  Results: s3://{FINE_TUNING_BUCKET}/{RESULTS_KEY}")
print(f"  Examples: s3://{FINE_TUNING_BUCKET}/{EXAMPLES_KEY}")

# Print summary
print("\n" + "=" * 60)
print("Summary")
print("=" * 60)
print(f"Run ID: {RUN_ID}")
print(f"Model: {MODEL_NAME} (LoRA fine-tuned)")
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"Training time: {training_time/60:.2f} minutes")
print(f"Peak memory: {peak_memory:.2f} MB")
print(f"\nText → Image:")
print(f"  R@1: {text_to_image_results.get('R@1', 0):.4f}")
print(f"  R@5: {text_to_image_results.get('R@5', 0):.4f}")
print(f"  R@10: {text_to_image_results.get('R@10', 0):.4f}")
if "MRR" in text_to_image_results:
    print(f"  MRR: {text_to_image_results['MRR']:.4f}")
print(f"\nImage → Text:")
print(f"  R@1: {image_to_text_results.get('R@1', 0):.4f}")
print(f"  R@5: {image_to_text_results.get('R@5', 0):.4f}")
print(f"  R@10: {image_to_text_results.get('R@10', 0):.4f}")
if "MRR" in image_to_text_results:
    print(f"  MRR: {image_to_text_results['MRR']:.4f}")
print(f"\n📁 All outputs saved to: s3://{FINE_TUNING_BUCKET}/{RUN_DIR}/")
